<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#TO-DO" data-toc-modified-id="TO-DO-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>TO-DO</a></span></li><li><span><a href="#Fragen" data-toc-modified-id="Fragen-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Fragen</a></span></li><li><span><a href="#Datenlücken" data-toc-modified-id="Datenlücken-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Datenlücken</a></span></li><li><span><a href="#Jährliche-Stromdaten" data-toc-modified-id="Jährliche-Stromdaten-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Jährliche Stromdaten</a></span></li><li><span><a href="#Strommix" data-toc-modified-id="Strommix-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Strommix</a></span></li><li><span><a href="#Primärenergie" data-toc-modified-id="Primärenergie-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Primärenergie</a></span></li><li><span><a href="#CO2-Emissionen" data-toc-modified-id="CO2-Emissionen-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>CO2 Emissionen</a></span></li><li><span><a href="#Stromimporte" data-toc-modified-id="Stromimporte-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Stromimporte</a></span></li></ul></div>

In [1]:
%pwd

'C:\\Users\\Marius\\Desktop\\Spezi\\Notebook'

In [2]:
#!jt -t chesterish -T -N -kl  


#!jt -l

# TO-DO
- Fixnanvalues ändern damit carbon_intensity_discharge_avg und carbon_intensity_import_avg nicht interpoliert werden
- Grafiken der historischen Daten
Darstellen wo sind die Lücken(jahre, anzahl datzenpunkte)
Entscheidung daten ausschlißen oder interpolieren
Carpetplot (Y: Stunden , X: tage, Wo sind die Löcher?)

- Endenergie mit Primärenergie vergleichen
- Imports genauer anschaune (Nach monat plotten)
- LIteratur komplett (Elec-Map,  was meint OIb mit CO2 inte, was meint ELec)(Wie wurden diese Daten erhoben)
- Literartur in Österreich
- Historische Analyse soweit fertig
- Methode bis skalierung
- Recherche für Verbrauch
- Abgabe als Word
- 26.11 9 Uhr 

# Fragen
- 70% Abgabe als notebook?
Als Text
- CO2/strommix werte von Importstrom auch anpassen?
Auch im StateoftheArt für wichtigste importeuer beschrieben
- Kopierte Funktionen (Urheberrecht?)
Für mich sollte kein Problem sein ()
- Kommentare/Variablennamen auf deutsch(konsistenz mit geschriebener sprache)
Egal aber konsistent
Grafiken auf jeden Fall auf Deutsch




In [3]:
%matplotlib inline
%matplotlib widget

In [4]:
from datetime import datetime 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import timezone
plt.rcParams.update({'figure.max_open_warning': 0})

import matplotlib
import locale
# Set to German locale to get comma decimal separater
locale.setlocale(locale.LC_NUMERIC, "deu_deu")

'German_Germany.1252'

In [5]:
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ ")
print("In the current dataframe df_2015 there are several rows with NAN-values ")
print("This function aims to fix these gaps by calculating a linear slope and extraploating the values (Geradengleichung) ")
print("Input:  df_2015, 'row of Dataframe','colum of Dataframe' ")
print("Output:'Fixed Dataframe'")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ ")
def FixNANValues(column,dfSource):
#if datapoint is empty we replace it with the average of the datapoint before and after 
    row = 0
    rowBeg = row 
    
    while row < len(dfSource.index):
        if dfSource.iloc[row,column] != dfSource.iloc[row,column]:
            rowBeg = row
            #Check how many empty datapoints there are
            while(dfSource.iloc[row,column] != dfSource.iloc[row,column]):
                row += 1
            #If Column is completely empty we return prematurely
            if len(dfSource.index) == row:
                return 
            slope = (dfSource.iloc[row,column]-dfSource.iloc[rowBeg-1,column])/((row-rowBeg)+1)
            while(rowBeg != row):
                #Insert interpolated Data into Dataframe
                dfSource.iloc[rowBeg,column] = dfSource.iloc[rowBeg-1,column] + slope
                rowBeg += 1
        row += 1
    return   

    

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ 
In the current dataframe df_2015 there are several rows with NAN-values 
This function aims to fix these gaps by calculating a linear slope and extraploating the values (Geradengleichung) 
Input:  df_2015, 'row of Dataframe','colum of Dataframe' 
Output:'Fixed Dataframe'
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ 


# Datenlücken

Die Daten aus der Electricitymap sind nicht ganz vollstänndig. Mithilfe einer Heatmap wurden diese Datenlücken visualisiert. Es treten in vier monaten Datenlücken, mit einer Größe bis maximal einem Tag, auf. Diese lücken treten spärlich verteilt auf, was eine lineare Interpolation genauer macht. Der schlimmste Fall ist in November 2019, wo ein gesamter Tag nicht aufgezeichnet wurde.


REchte Seite in Prozent statt absolut
Alle Jahre als band darstellen.

In [6]:
fig, ax = plt.subplots(6,2,figsize=(10,12),gridspec_kw={'width_ratios': [2, 1]})

IsNaN = pd.DataFrame()

li = ["Mai","Juni","September","November"]

df_2015 = pd.read_csv("../Data/Electricity_map_CO2_AT_2015_2017.csv"
                 ,delimiter = ';',parse_dates=["datetime"],index_col=["datetime"])
df_IsNaN = pd.DataFrame()
df_IsNaN["IsNaN"] = df_2015["carbon_intensity_avg"].isin([np.nan])
df_IsNaN = df_IsNaN.transpose() 


sns.heatmap(df_IsNaN , ax = ax[0][0], cbar = 0 , yticklabels= False, cmap = ["black","cyan"])
ax[0][0].set_xlabel("")
ax[0][0].set_xticks([0,8760,17520,26280])
ax[0][0].set_xticklabels(["2015","2016","2017","2018"],rotation='horizontal')
ax[0][0].set_title("2015 - 2018")

df_IsNaN = df_IsNaN.transpose()

ax[0][1].pie(df_IsNaN["IsNaN"].value_counts(),  autopct='%1.1f%%',pctdistance = 1.7, radius = 1.3,
         startangle=0, colors = ["black","cyan"])
##############################################################################################
df_2018 = pd.read_csv("../Data/Electricity_map_CO2_AT_2018_2019.csv"
                 ,delimiter = ';',parse_dates=["datetime"],index_col=["datetime"])
df_IsNaN = pd.DataFrame()
df_IsNaN["IsNaN"] = df_2018["carbon_intensity_avg"].isin([np.nan])
df_IsNaN = df_IsNaN.transpose() 


sns.heatmap(df_IsNaN , ax = ax[1][0], cbar = 0 , yticklabels= False, cmap = ["black","cyan"])
ax[1][0].set_xlabel("")
ax[1][0].set_xticks([0,8760,17520])
ax[1][0].set_xticklabels(["2018","2019","2020"],rotation='horizontal')
ax[1][0].set_title("2018 - 2020")

df_IsNaN = df_IsNaN.transpose()

ax[1][1].pie(df_IsNaN["IsNaN"].value_counts(),  autopct='%1.1f%%',pctdistance = 1.7, radius = 1.3,
         startangle=0, colors = ["black","cyan"])

###################################################################################################
for i,month in enumerate([5,6],2):
    df_2015 = pd.read_csv("../Data/Electricity_map_CO2_AT_2015_2017.csv"
                     ,delimiter = ';',parse_dates=["datetime"],index_col=["datetime"])
    df_2015 = df_2015[df_2015.index.year == 2016]
    df_2015 = df_2015[df_2015.index.month == month]
    df_IsNaN = pd.DataFrame()
    df_IsNaN["IsNaN"] = df_2015["carbon_intensity_avg"].isin([np.nan])
    df_IsNaN = df_IsNaN.transpose() 

    
    sns.heatmap(df_IsNaN , ax = ax[i][0], cbar = 0 , yticklabels= False, cmap = ["black","cyan"])
    ax[i][0].set_xlabel("Woche")
    ax[i][0].set_xticks([0,168,336,504,672])
    ax[i][0].set_xticklabels(["1","2","3","4","5"],rotation='horizontal')
    ax[i][0].set_title( li[i-2] + " 2016")
    
    df_IsNaN = df_IsNaN.transpose()
    
    ax[i][1].pie(df_IsNaN["IsNaN"].value_counts(),  autopct='%1.1f%%',pctdistance = 1.7, radius = 1.3,
         startangle=0, colors = ["black","cyan"])

    
IsNaN = pd.DataFrame()



for i,month in enumerate([9,11],4):
    df_Temp = pd.read_csv("../Data/Electricity_map_CO2_AT_2018_2019.csv"
                     ,delimiter = ';',parse_dates=["datetime"],index_col=["datetime"])
    df_Temp = df_Temp[df_Temp.index.year == 2019]
    df_Temp = df_Temp[df_Temp.index.month == month]
    df_IsNaN = pd.DataFrame()
    df_IsNaN["IsNaN"] = df_Temp["carbon_intensity_avg"].isin([np.nan])
    df_IsNaN = df_IsNaN.transpose() 

    sns.heatmap(df_IsNaN , ax = ax[i][0], cbar = 0 , yticklabels= False, cmap = ["black","cyan"])
    ax[i][0].set_xlabel("Woche")
    ax[i][0].set_xticks([0,168,336,504,672])
    ax[i][0].set_xticklabels(["1","2","3","4","5"],rotation='horizontal')
    ax[i][0].set_title( li[i-2] + " 2019")
    
    df_IsNaN = df_IsNaN.transpose()
    
    
    ax[i][1].pie(df_IsNaN["IsNaN"].value_counts(),  autopct='%1.1f%%',pctdistance = 1.7, radius = 1.3,
         startangle=0, colors = ["black","cyan"])

fig.suptitle('Auswertung der Datenlücken', fontsize=20)
leg = fig.legend(labels = ["Daten vorhanden","Datenlücken"], bbox_to_anchor = [0.74, 0.945],
            prop={'size': 14},ncol = 3,facecolor='#f5f5f5', framealpha=1)

leg.legendHandles[0].set_color('black')
leg.legendHandles[1].set_color('cyan')

#Move other axis out of the way
pos = ax[0][0].get_position()
pos.y0 = 0.98    #height
ax[0][0].set_position(pos)   
    
    
plt.tight_layout()
plt.savefig("../Bilder/Datenlücken", bbox_inches = 'tight',pad_inches = 0.08)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
df_2015.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 720 entries, 2016-06-01 00:00:00+00:00 to 2016-06-30 23:00:00+00:00
Data columns (total 18 columns):
 #   Column                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         Non-Null Count  Dtype  
---  ------                                                                                                                                                                                                                                                                                                                                       

In [8]:
df_2018.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 17505 entries, 2018-01-01 00:00:00+00:00 to 2019-12-31 23:00:00+00:00
Data columns (total 83 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   Jahr                                          17505 non-null  int64  
 1   monat                                         17505 non-null  int64  
 2   Tag                                           17505 non-null  int64  
 3   Stunde                                        17505 non-null  int64  
 4   Datum                                         17505 non-null  object 
 5   Tag des Jahres                                17505 non-null  int64  
 6   Tag des Monats                                17505 non-null  int64  
 7   Uhrzeit                                       17505 non-null  object 
 8   timestamp                                     17505 non-null  int64  
 9   zone_name     

In [9]:
df_2015 = pd.read_csv("../Data/Electricity_map_CO2_AT_2015_2017.csv"
                     ,delimiter = ';',parse_dates=["datetime"],index_col=["datetime"])

for column in range(0,18,1):
    FixNANValues(column,df_2015)        

li_energySource = ["power_consumption_biomass_avg","power_consumption_coal_avg","power_consumption_gas_avg",
                    "power_consumption_hydro_avg","power_consumption_nuclear_avg","power_consumption_oil_avg",
                    "power_consumption_solar_avg","power_consumption_wind_avg","power_consumption_geothermal_avg",
                    "power_consumption_unknown_avg","power_consumption_hydro_discharge_avg"]
li_energyPercent = ["power_origin_percent_biomass_avg","power_origin_percent_coal_avg","power_origin_percent_gas_avg",
                    "power_origin_percent_hydro_avg","power_origin_percent_nuclear_avg","power_origin_percent_oil_avg",
                    "power_origin_percent_solar_avg","power_origin_percent_wind_avg","power_origin_percent_geothermal_avg",
                    "power_origin_percent_unknown_avg","power_origin_percent_hydro_discharge_avg"]

for column in range(0,11,1):
    df_2015[li_energySource[column]] = df_2015["total_production_avg"] \
                                    .multiply(df_2015[li_energyPercent[column]]).divide(100)
      

df_2015["power_origin_percent_fossil_avg"] = df_2015["power_origin_percent_coal_avg"] + df_2015["power_origin_percent_gas_avg"] + \
                                            df_2015["power_origin_percent_nuclear_avg"] + df_2015["power_origin_percent_oil_avg"] + \
                                            df_2015["power_origin_percent_unknown_avg"]

df_2015["power_origin_percent_renewable_avg"] = df_2015["power_origin_percent_biomass_avg"] + df_2015["power_origin_percent_hydro_avg"] + \
                                                df_2015["power_origin_percent_solar_avg"] + df_2015["power_origin_percent_wind_avg"] + \
                                                df_2015["power_origin_percent_geothermal_avg"] + df_2015["power_origin_percent_hydro_discharge_avg"]

df_2015.to_csv("../Data/Electricity_map_CO2_AT_2015_2017_fixed.csv",sep = ';')


In [10]:
#creates pandas object to read csv
df_2018 = pd.read_csv("../Data/Electricity_map_CO2_AT_2018_2019.csv"
                      ,delimiter = ';',parse_dates=["datetime"],index_col=["datetime"])
   
columnNames = list(df_2018.columns)
columnNum = 0
for column in columnNames:   
    if isinstance(df_2018[column].iloc[2], str) or df_2018[column].dropna().empty:
        columnNum += 1
        continue
    FixNANValues(columnNum,df_2018)
    columnNum += 1   
    
    
li_energySource = ["carbon_production_biomass_avg","carbon_production_coal_avg","carbon_production_gas_avg",
                    "carbon_production_hydro_avg","carbon_production_nuclear_avg","carbon_production_oil_avg",
                    "carbon_production_solar_avg","carbon_production_wind_avg","carbon_production_geothermal_avg",
                    "carbon_production_unknown_avg", "carbon_production_battery_discharge_avg", 
                   "carbon_production_hydro_discharge_avg"]
li_energyPercent = ["carbon_origin_percent_biomass_avg","carbon_origin_percent_coal_avg","carbon_origin_percent_gas_avg",
                    "carbon_origin_percent_hydro_avg","carbon_origin_percent_nuclear_avg","carbon_origin_percent_oil_avg",
                    "carbon_origin_percent_solar_avg","carbon_origin_percent_wind_avg","carbon_origin_percent_geothermal_avg",
                    "carbon_origin_percent_unknown_avg","carbon_origin_percent_battery_discharge_avg",
                    "carbon_origin_percent_hydro_discharge_avg"]

for column in range(0,12,1):
    df_2018[li_energySource[column]] = df_2018["carbon_intensity_avg"] \
                                    .multiply(df_2018[li_energyPercent[column]]).divide(100)    
    
    
    
dfToPlot = pd.DataFrame()

df_2018["power_origin_percent_fossil_avg"] = (df_2018["power_consumption_coal_avg"] + df_2018["power_consumption_gas_avg"] +\
                                             df_2018["power_consumption_nuclear_avg"] + df_2018["power_consumption_oil_avg"] + \
                                            df_2018["power_consumption_unknown_avg"]) / df_2018["total_consumption_avg"] *100

df_2018["power_origin_percent_renewable_avg"] = (df_2018["power_consumption_biomass_avg"] + df_2018["power_consumption_hydro_avg"] +\
                                             df_2018["power_consumption_solar_avg"] + df_2018["power_consumption_wind_avg"] + \
                                            df_2018["power_consumption_geothermal_avg"] + df_2018["power_consumption_hydro_discharge_avg"]) \
                                            / df_2018["total_consumption_avg"] *100


li_consumption_percent_src = ["power_consumption_biomass_avg","power_consumption_coal_avg","power_consumption_gas_avg",
                         "power_consumption_hydro_avg","power_consumption_nuclear_avg","power_consumption_oil_avg",
                         "power_consumption_solar_avg","power_consumption_wind_avg","power_consumption_geothermal_avg",
                         "power_consumption_unknown_avg","power_consumption_hydro_discharge_avg",]

li_consumption_percent_target = ["power_origin_percent_biomass_avg","power_origin_percent_coal_avg","power_origin_percent_gas_avg",
                         "power_origin_percent_hydro_avg","power_origin_percent_nuclear_avg","power_origin_percent_oil_avg",
                         "power_origin_percent_solar_avg","power_origin_percent_wind_avg","power_origin_percent_geothermal_avg",
                         "power_origin_percent_unknown_avg","power_origin_percent_hydro_discharge_avg",]

for i in range(0,len(li_consumption_percent_target),1):
    df_2018[li_consumption_percent_target[i]] =df_2018[li_consumption_percent_src[i]] / df_2018["total_consumption_avg"] *100

df_2018.to_csv("../Data/Electricity_map_CO2_AT_2018_2019_fixed.csv",sep = ';')

In [11]:
df_2015.info(verbose = True)

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 26304 entries, 2015-01-01 00:00:00+00:00 to 2017-12-31 23:00:00+00:00
Data columns (total 31 columns):
 #   Column                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         Non-Null Count  Dtype  
---  ------                                                                                                                                                                                                                                                                                                                                     

In [12]:
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ ")
print("This cell combines the shared columns of df_2015 and df_2018")
print("Input: df_2015, df_2018 ")
print("Output: df_2015_2019")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ ")
df_2015_2019 = pd.DataFrame()
df_temp1 = pd.DataFrame()
df_temp2 = pd.DataFrame()
i = 0
for column in df_2015:
    while i != 106:  
        if column == df_2018.columns[i]:
            df_temp1[column] = df_2015[column]
            df_temp2[column] = df_2018[column]
            df_2015_2019 = pd.concat([df_temp1,df_temp2])
        i += 1
    i = 0

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ 
This cell combines the shared columns of df_2015 and df_2018
Input: df_2015, df_2018 
Output: df_2015_2019
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ 


In [13]:
df_2015_2019.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 43809 entries, 2015-01-01 00:00:00+00:00 to 2019-12-31 23:00:00+00:00
Data columns (total 29 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   total_production_avg                      43809 non-null  float64
 1   total_consumption_avg                     43809 non-null  float64
 2   total_import_avg                          43809 non-null  float64
 3   total_export_avg                          43809 non-null  float64
 4   carbon_intensity_avg                      43809 non-null  float64
 5   power_origin_percent_biomass_avg          43809 non-null  float64
 6   power_origin_percent_coal_avg             43809 non-null  float64
 7   power_origin_percent_gas_avg              43809 non-null  float64
 8   power_origin_percent_hydro_avg            43809 non-null  float64
 9   power_origin_percent_nuclear_avg          43809 non-null  

In [14]:
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ ")
print("There is currently no clean way to add the weeknumber since index.week is deprecated")
print("This function creates them with a loop and the funciton isocalendar")
print("Name: CreateWeekNum ")
print("Input: df_Source ")
print("Output: list of weeknumbers")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ ")
def CreateWeekNum(df_Source):
    li_weeknum = []
    for rows in range(0,len(df_Source),1):
        li_weeknum.append(df_Source.index[rows].isocalendar()[1])
    return li_weeknum    


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ 
There is currently no clean way to add the weeknumber since index.week is deprecated
This function creates them with a loop and the funciton isocalendar
Name: CreateWeekNum 
Input: df_Source 
Output: list of weeknumbers
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ 


In [15]:
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ ")
print("Little helper function to add various time related columns to dataframes")
print("Name: DataFrameTimeIndexing ")
print("Input: df_Source ")
print("Output: added columns to DataFrame")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ ")
def DataFrameTimeIndexing(dfSource):
    dfSource['year'] = dfSource.index.year
    dfSource['quarter'] = dfSource.index.quarter
    dfSource['month'] = dfSource.index.month
    dfSource['week'] = CreateWeekNum(dfSource)
    dfSource['date'] = dfSource.index.day
    dfSource['hour'] = dfSource.index.hour
    return
    
DataFrameTimeIndexing(df_2015_2019)
DataFrameTimeIndexing(df_2015)
DataFrameTimeIndexing(df_2018)    

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ 
Little helper function to add various time related columns to dataframes
Name: DataFrameTimeIndexing 
Input: df_Source 
Output: added columns to DataFrame
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ 


# Jährliche Stromdaten

In [16]:
fig, ax = plt.subplots(2,2,figsize=(10,6))

li_ToPLot = ["total_production_avg","total_consumption_avg","total_import_avg","total_export_avg"]
li_Title = ["Produktion","Verbrauch","Import","Export"]

i = 0
for row in range(0,2,1):
    for col in range(0,2,1):
        
        sns.barplot(x = "year", y = li_ToPLot[i], data = df_2015_2019.resample("Y").sum().divide(1e6), 
                    ax = ax[row][col])
        
        ax[row][col].set_title(li_Title[i])
        ax[row][col].set_ylabel("Energie\n[TWh]")
        ax[row][col].set_xlabel("")
        ax[row][col].set_xticklabels(["2015","2016","2017","2018","2019"])
        #adjust ylim so we can see the numbers
        lim = ax[row][col].get_ylim()
        ax[row][col].set_ylim(0,lim[1]+lim[1]/3)
        #write the values ontop of the axes
        for p in ax[row][col].patches:
                 ax[row][col].annotate("%.2f" % p.get_height(), (p.get_x() + p.get_width() / 2., p.get_height()),
                     ha='center', va='center', fontsize=13, color='black', xytext=(0, 10),
                     textcoords='offset points')
        i += 1
            
fig.suptitle('Jährliche Stromdaten für die\nJahre 2015-2020', fontsize=20)
#Move other axis out of the way
pos = ax[0][0].get_position()
pos.y1 = 0.98    #height
ax[0][0].set_position(pos)
plt.tight_layout()
plt.savefig("../Bilder/Jährliche Stromdaten für die Jahre 2015-2020", bbox_inches = 'tight',pad_inches = 0.08)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
df_2018.info(verbose =True)

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 17505 entries, 2018-01-01 00:00:00+00:00 to 2019-12-31 23:00:00+00:00
Data columns (total 112 columns):
 #   Column                                        Dtype  
---  ------                                        -----  
 0   Jahr                                          int64  
 1   monat                                         int64  
 2   Tag                                           int64  
 3   Stunde                                        int64  
 4   Datum                                         object 
 5   Tag des Jahres                                int64  
 6   Tag des Monats                                int64  
 7   Uhrzeit                                       object 
 8   timestamp                                     int64  
 9   zone_name                                     object 
 10  carbon_intensity_avg                          float64
 11  carbon_intensity_production_avg               float64
 12  carbon_inten

In [18]:
#total_discharge_avg

df_2018["power_net_discharge_hydro_avg"].clip(0,999999).sum() /1000

8959.894576922054

In [19]:
fig, ax = plt.subplots(2,4,figsize=(10,6))

li_ToPLot = ["power_production_hydro_avg","power_production_coal_avg","power_production_gas_avg",
            "power_production_biomass_avg","power_production_wind_avg","power_production_solar_avg",
             "power_net_discharge_hydro_avg"]
li_Title = ["Wasser","Kohle","Gas","Biomasse","Wind","Photovoltaik","Pumpspeicher-\nkraftwerke"]

i = 0
for col in range(0,4,1):
    for row in range(0,2,1):
        if i == 7:
            ax[row][col].remove()
            continue
        sns.barplot(x = "year", y = li_ToPLot[i], data = df_2018.resample("Y").sum().divide(1e6), 
                    ax = ax[row][col], palette=["C3","C4"])
        
        ax[row][col].set_title(li_Title[i])
        ax[row][col].set_ylabel("Energie\n[TWh]")
        ax[row][col].set_xlabel("")
        ax[row][col].set_xticklabels(["2018","2019"])
        #adjust ylim so we can see the numbers
        lim = ax[row][col].get_ylim()
        ax[row][col].set_ylim(0,lim[1]+lim[1]/3)
        #write the values ontop of the axes
        for p in ax[row][col].patches:
                 ax[row][col].annotate("%.2f" % p.get_height(), (p.get_x() + p.get_width() / 2., p.get_height()),
                     ha='center', va='center', fontsize=13, color='black', xytext=(0, 10),
                     textcoords='offset points')
        i += 1
            
fig.suptitle('Jährliche Stromdaten für\neinzelne Energieträger', fontsize=20)
#Move other axis out of the way
pos = ax[0][0].get_position()
pos.y1 = 0.98    #height
ax[0][0].set_position(pos)
plt.tight_layout()
plt.savefig("../Bilder/Jährliche Stromdaten für einzelne Energieträgergruppen", bbox_inches = 'tight',pad_inches = 0.08)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [20]:
# Quelle: https://www.oib.or.at/sites/default/files/294145_oib-rl_6_entwurf_juni_2018.pdf
OIB_RL6 = [
    ["Jänner"   , 1.98, 1.46, 0.52, 332],
    ["Februar"  , 1.97, 1.42, 0.55, 322],
    ["März"     , 1.89, 1.28, 0.61, 288],
    ["April"    , 1.73, 1.03, 0.7, 230],
    ["Mai"      , 1.61, 0.85, 0.76, 182],
    ["Juni"     , 1.60, 0.83, 0.77, 179],
    ["Juli"     , 1.58, 0.82, 0.76, 178],
    ["August"   , 1.62, 0.85, 0.77, 182],
    ["September", 1.73, 1.03, 0.70, 227],
    ["Oktober"  , 1.88, 1.26, 0.62, 284],
    ["November" , 1.94, 1.38, 0.56, 308],
    ["Dezember" , 1.96,1.42, 0.54, 318]  
]
df_OIB_RL6 = pd.DataFrame(OIB_RL6, columns = ["month","fPE","fPE,n.ern", "fPE,ern" , "fCO2eq" ])


OE_NORM = [
    [2.144,0.066,1.013,-1.969,0.352,-3.294,-3.79,-4.233,-0.731,-5.349,0.382,-0.981,0.71,-2.388],
    [3.918,2.972,3.088,1.366,2.181,-0.184,2.260,0.046,2.22,0.035,2.706,2.045,2.549,0.834],
    [7.793,7.411,7.333,6.402,6.523,4.943,6.940,5.696,7.226,6.448,7.137,6.814,6.762,5.945],
    [12.511,11.99,12.416,11.532,11.802,10.533,12.16,11.289,12.482,12.253,12.236,11.895,11.872,11.369],
    [16.576,16.013,16.652,15.874,16.220,15.175,16.462,15.772,16.841,16.84,16.666,16.362,16.306,15.881],
    [20.023,19.179,20.047,18.961,19.63,18.308,20.071,18.95,20.619,20.516,20.245,19.923,19.855,19.196],
    [21.92,20.94,21.76,20.24,21.5,19.69,21.74,20.19,22.47,22.01,21.97,21.56,21.84,20.87],
    [21.44,19.86,21.26,19.37,20.94,18.78,21.18,19.3,21.7,20.67,21.24,20.67,21.27,19.9],
    [17.6,16.44,17.45,15.93,16.94,14.91,17.29,15.73,17.57,16.63,17.37,16.92,17.28,16.13],
    [12.08,10.54,11.75,9.72,11.03,8.41,11.5,9.18,11.57,9.5,11.62,11.02,11.38,9.576],
    [7.19,5.01,6.49,3.73,5.64,2.43,5.87,2.66,5.6,2.22,5.98,4.93,5.94,3.34],
    [3.88,0.29,2.86,-1.46,2,-2.65,1.69,-3.49,1.15,-4.66,1.99,0.144,2.26,-1.89]  
]


# Strommix

Schauen warum die Änderungen auf einmal so abfallen im jahr 2016(Ob irgendein Gesetz aufgelaufenist vielleicht)


In [21]:
df_2015_2019.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 43809 entries, 2015-01-01 00:00:00+00:00 to 2019-12-31 23:00:00+00:00
Data columns (total 35 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   total_production_avg                      43809 non-null  float64
 1   total_consumption_avg                     43809 non-null  float64
 2   total_import_avg                          43809 non-null  float64
 3   total_export_avg                          43809 non-null  float64
 4   carbon_intensity_avg                      43809 non-null  float64
 5   power_origin_percent_biomass_avg          43809 non-null  float64
 6   power_origin_percent_coal_avg             43809 non-null  float64
 7   power_origin_percent_gas_avg              43809 non-null  float64
 8   power_origin_percent_hydro_avg            43809 non-null  float64
 9   power_origin_percent_nuclear_avg          43809 non-null  

In [22]:
dfToPlot = pd.DataFrame()

dfToPlot["total_consumption_avg"] = df_2015_2019["total_consumption_avg"].resample("Y").sum()
dfToPlot

,total_consumption_avg
datetime,
2015-12-31 00:00:00+00:00,6.018961e+07
2016-12-31 00:00:00+00:00,6.088008e+07
2017-12-31 00:00:00+00:00,6.345902e+07
2018-12-31 00:00:00+00:00,6.349759e+07
2019-12-31 00:00:00+00:00,5.992832e+07


In [23]:
fig, ax = plt.subplots(figsize=(12, 5))
dfToPlot = pd.DataFrame()

dfToPlot["total_consumption_avg"] = df_2015_2019["total_consumption_avg"].resample("Y").sum()

#dfToPlot =  dfToPlot.resample("W").mean()
x = np.arange(0,len(dfToPlot),1)
z = np.polyfit(x = x, y = dfToPlot["total_consumption_avg"],deg = 1)
p = np.poly1d(z)
plt.plot(x,p(x),"r-")
li_Trend = []
for i in range(0,262,1):
    li_Trend.append(p(i))
p(15) / 1e6

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

64.31433748123489

In [24]:
fig, ax = plt.subplots(figsize=(12, 5))
#labels = dfToPlot.index
dfToPlot = pd.DataFrame()
dfToPlot["power_origin_percent_fossil_avg"] = df_2015_2019["power_origin_percent_fossil_avg"]
dfToPlot["power_origin_percent_renewable_avg"] = df_2015_2019["power_origin_percent_renewable_avg"]

dfToPlot =  dfToPlot.resample("W").mean()

dfToPlot.plot(kind="area",linewidth=0, color=["grey","green"], stacked=True, ax=ax, legend=False);

x = np.arange(0,len(dfToPlot),1)
z = np.polyfit(x = x, y = dfToPlot["power_origin_percent_fossil_avg"],deg = 1)
p = np.poly1d(z)
li_Trend = []
for i in range(0,262,1):
    li_Trend.append(p(i))
dfToPlot["Trend"] = li_Trend
dfToPlot["Trend"].plot(kind = "line", ax = ax, color = "orange")


ax.set_xlabel('')
ax.set_ylabel('Prozent [%]', fontsize=14)

ax.yaxis.set_tick_params(labelsize=14)
ax.xaxis.set_tick_params(labelsize=14)

fig.suptitle('Strommix der Jahre 2015-2020', fontsize=20, x = 0.5,y = 0.98)
#create a legend for the whole figure
leg = fig.legend(labels = ["Anteil nicht erneuerbar","Anteil erneuerbar","Trendlinie des fossilen Anteils"], 
                 bbox_to_anchor = [0.85, 0.94],  prop={'size': 12},facecolor='#f5f5f5', framealpha=1, ncol = 3)
#change width of legend color lines
for legobj in leg.legendHandles:
    legobj.set_linewidth(5)
#Move other axis out of the way
pos = ax.get_position()
pos.y1 = 1.    #height
ax.set_position(pos)

plt.tight_layout()
plt.savefig("../Bilder/Strommix_der_Jahre2018-2019", bbox_inches = 'tight',pad_inches = 0.08)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [25]:
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ ")
print("stellt die stromprouktion über die Jahre aufgeteilt auf die einzelnen energieträger dar")
print("Bin mir aber ziemlich sicher dass die Power_origin_percent auf verbrauchten strom deuten")
print("Siehe anteil an Nuklear sowie großer Kohleanteil")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ ")


colors = ["green", "black","purple", "#0088ff", "#b4b88a", "brown", "orange", "cyan","grey","#0000ff","red"]



li_ToPLot = ["power_consumption_biomass_avg","power_consumption_coal_avg",
            "power_consumption_gas_avg","power_consumption_hydro_avg",
            "power_consumption_nuclear_avg","power_consumption_oil_avg",
            "power_consumption_solar_avg","power_consumption_wind_avg",
            "power_consumption_geothermal_avg","power_consumption_hydro_discharge_avg",
             "power_consumption_unknown_avg",]

li_Legend = ["Biomasse","Kohle","Gas","Wasser","Nuklear","Öl","Photovoltaik",
             "Wind","Geothermie","Speicherwasser","Unbekannt","Trendlinie"]

fig, ax = plt.subplots(figsize=(12, 5))
#labels = dfToPlot.index
df_2015_2019_weekly = df_2015_2019[li_ToPLot].resample("M").mean()
dfToPlot = df_2015_2019_weekly[li_ToPLot]
dfToPlot.plot(kind = "area",ax=ax, color = colors )

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
dfToPlot = pd.DataFrame()

dfToPlot["total_consumption_avg"] = df_2015_2019["total_consumption_avg"]

dfToPlot =  dfToPlot.resample("M").mean()
x = np.arange(0,len(dfToPlot),1)
z = np.polyfit(x = x, y = dfToPlot["total_consumption_avg"],deg = 1)
p = np.poly1d(z)
li_Trend = []
for i in range(0,len(dfToPlot),1):
    li_Trend.append(p(i))
dfToPlot["Trend"] = li_Trend
dfToPlot["Trend"].plot(kind = "line", ax = ax, color = "#1e5202")
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
leg = ax.legend(bbox_to_anchor=(0.47,1.35), title = "Energieträger", 
                labels = li_Legend, ncol = 6, loc = "upper center",
               facecolor='#f5f5f5', framealpha=1)

ax.set_xlabel('')
ax.set_ylabel('Stromverbrauch [MW]', fontsize = 14)
fig.suptitle('Aufgeteilter Strommix der Jahre 2015-2020', fontsize=20, x = 0.5)

for legobj in leg.legendHandles:
    legobj.set_linewidth(5.0)


#Move other axis out of the way
pos = ax.get_position()
pos.y1 = 0.88  #height
ax.set_position(pos)
ax.get_yaxis().set_major_formatter(matplotlib.ticker.FuncFormatter(lambda x, loc: locale.format_string('%d', x, 1)))
plt.tight_layout()
plt.savefig("../Bilder/Stromproduktion", bbox_inches = 'tight',pad_inches = 0.08)


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ 
stellt die stromprouktion über die Jahre aufgeteilt auf die einzelnen energieträger dar
Bin mir aber ziemlich sicher dass die Power_origin_percent auf verbrauchten strom deuten
Siehe anteil an Nuklear sowie großer Kohleanteil
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ 


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [26]:
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ ")
print("stellt die stromprouktion über die Jahre aufgeteilt auf die einzelnen energieträger dar")
print("Bin mir aber ziemlich sicher dass die Power_origin_percent auf verbrauchten strom deuten")
print("Siehe anteil an Nuklear sowie großer Kohleanteil")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ ")


colors = ["green", "black","purple", "#0088ff", "#b4b88a", "brown", "orange", "cyan","grey","#0000ff","red"]



li_ToPLot = ["power_consumption_biomass_avg","power_consumption_coal_avg",
            "power_consumption_gas_avg","power_consumption_hydro_avg",
            "power_consumption_nuclear_avg","power_consumption_oil_avg",
            "power_consumption_solar_avg","power_consumption_wind_avg",
            "power_consumption_geothermal_avg","power_consumption_hydro_discharge_avg",
             "power_consumption_unknown_avg",]

li_Legend = ["Biomass","Coal","Gas","Water","Nuclear","Oil","Photovoltaic",
             "Wind","Geothermal","PSP","Unknown"]

fig, ax = plt.subplots(figsize=(12, 5))
#labels = dfToPlot.index
df_2015_2019_weekly = df_2015_2019[li_ToPLot].resample("M").mean()
dfToPlot = df_2015_2019_weekly[li_ToPLot]
dfToPlot.plot(kind = "area",ax=ax, color = colors )
leg = ax.legend(bbox_to_anchor=(1,1), title = "Energy source", 
                labels = li_Legend)

ax.set_xlabel('')
ax.set_ylabel('Consumed electricity [MW]', fontsize = 14)
fig.suptitle('Electricity mix of the years 2015-2020', fontsize=20, x = 0.45)

for legobj in leg.legendHandles:
    legobj.set_linewidth(5.0)


#Move other axis out of the way
pos = ax.get_position()
pos.y1 = 0.95   #height
ax.set_position(pos)

plt.tight_layout()
plt.savefig("../Bilder/Stromproduktion", bbox_inches = 'tight',pad_inches = 0.08)

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ 
stellt die stromprouktion über die Jahre aufgeteilt auf die einzelnen energieträger dar
Bin mir aber ziemlich sicher dass die Power_origin_percent auf verbrauchten strom deuten
Siehe anteil an Nuklear sowie großer Kohleanteil
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ 


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [27]:
fig, ax = plt.subplots(2,2, figsize=(12.5,8),gridspec_kw={'width_ratios': [3, 1]})

li_Carbon_RE = ["power_consumption_biomass_avg","power_consumption_hydro_avg","power_consumption_solar_avg",
               "power_consumption_wind_avg","power_consumption_geothermal_avg","power_consumption_hydro_discharge_avg"]

li_Carbon_NRE = ["power_consumption_coal_avg","power_consumption_gas_avg","power_consumption_nuclear_avg",
                "power_consumption_oil_avg","power_consumption_unknown_avg"]
               

li_Colors_RE = ["green", "#0088ff", "orange", "grey" , "brown", "#0000ff"]

li_Colors_NRE = ["grey", "purple", "#b4b88a", "brown" , "red"]

dfToPlot = pd.DataFrame()
for i in range(0,len(li_Carbon_RE),1):
    dfToPlot[li_Carbon_RE[i]] = df_2015_2019[li_Carbon_RE[i]]
sns.boxplot(palette=li_Colors_RE,  data=dfToPlot, ax = ax[0][0])

sns.boxplot(y = "power_consumption_geothermal_avg",palette=["grey"], 
              data=dfToPlot, ax = ax[0][1])

ax[0][0].set_xticklabels(["Biomasse","Wasserkraft","Photovolatik","Windkraft","Geothermie","Pumpspeicher"],fontsize=14)
ax[0][1].set_xticklabels(["Geothermie"],fontsize=14)
ax[0][0].get_yaxis().set_major_formatter(matplotlib.ticker.FuncFormatter(lambda x, loc: locale.format_string('%d', x, 1)))

dfToPlot = pd.DataFrame()
for i in range(0,len(li_Carbon_NRE),1):
    dfToPlot[li_Carbon_NRE[i]] = df_2015_2019[li_Carbon_NRE[i]]
    
sns.boxplot(palette=li_Colors_NRE, data=dfToPlot, ax = ax[1][0])

dfToPlot = dfToPlot.drop(["power_consumption_coal_avg","power_consumption_gas_avg"
                          ,"power_consumption_unknown_avg","power_consumption_nuclear_avg"], axis = 1)

sns.boxplot(palette=["brown"], 
              data=dfToPlot, ax = ax[1][1])

ax[1][0].set_xticklabels(["Kohle","Gas","Nuklear","Öl","Unbekannt"],fontsize=14)
ax[1][1].set_xticklabels(["Öl"],fontsize=14)
ax[1][0].get_yaxis().set_major_formatter(matplotlib.ticker.FuncFormatter(lambda x, loc: locale.format_string('%d', x, 1)))

for i in range(0,2,1):
    for j in range(0,2,1):
        ax[j][i].set_ylabel("Verbrauchter Strom [MW]", fontsize = 14)

#change width of legend color lines
for legobj in leg.legendHandles:
    legobj.set_linewidth(3.0)
    
fig.suptitle('Verteilung des Stromverbrauches für erneuerbare\n sowie nicht erneuerbare Energieträger', fontsize=20)

#Move other axis out of the way
pos = ax[0][0].get_position()
pos.y1 = 0.99    #height
ax[0][0].set_position(pos)

plt.tight_layout()
plt.savefig("../Bilder/Verteilung Stromverbrauch Energieträger", bbox_inches = 'tight',pad_inches = 0.08)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [28]:
fig, ax = plt.subplots(2,2, figsize=(12.5,8),gridspec_kw={'width_ratios': [3, 1]})



li_Carbon_RE = ["power_origin_percent_biomass_avg","power_origin_percent_hydro_avg","power_origin_percent_solar_avg",
               "power_origin_percent_wind_avg","power_origin_percent_geothermal_avg","power_origin_percent_hydro_discharge_avg"]

li_Carbon_NRE = ["power_origin_percent_coal_avg","power_origin_percent_gas_avg","power_origin_percent_nuclear_avg",
                "power_origin_percent_oil_avg","power_origin_percent_unknown_avg"]
               

li_Colors_RE = ["green", "#0088ff", "orange", "grey" , "brown", "#0000ff"]

li_Colors_NRE = ["grey", "purple", "#b4b88a", "brown" , "red"]

dfToPlot = pd.DataFrame()
for i in range(0,len(li_Carbon_RE),1):
    dfToPlot[li_Carbon_RE[i]] = df_2015_2019[li_Carbon_RE[i]]
sns.boxplot(palette=li_Colors_RE,  data=dfToPlot, ax = ax[0][0])

sns.boxplot(y = "power_origin_percent_geothermal_avg",palette=["grey"], 
              data=dfToPlot, ax = ax[0][1])

ax[0][0].set_xticklabels(["Biomasse","Wasserkraft","Photovolatik","Windkraft","Geothermie","Pumpspeicher"],fontsize=14)
ax[0][1].set_xticklabels(["Geothermie"],fontsize=14)

dfToPlot = pd.DataFrame()
for i in range(0,len(li_Carbon_NRE),1):
    dfToPlot[li_Carbon_NRE[i]] = df_2015_2019[li_Carbon_NRE[i]]
    
sns.boxplot(palette=li_Colors_NRE, data=dfToPlot, ax = ax[1][0])

dfToPlot = dfToPlot.drop(["power_origin_percent_coal_avg","power_origin_percent_gas_avg"
                          ,"power_origin_percent_unknown_avg","power_origin_percent_nuclear_avg"], axis = 1)

sns.boxplot(palette=["brown"], 
              data=dfToPlot, ax = ax[1][1])

ax[1][0].set_xticklabels(["Kohle","Gas","Nuklear","Öl","Unbekannt"],fontsize=14)
ax[1][1].set_xticklabels(["Öl"],fontsize=14)


for i in range(0,2,1):
    for j in range(0,2,1):
        ax[j][i].set_ylabel("Verbrauchter Strom [%]", fontsize = 14)

#change width of legend color lines
for legobj in leg.legendHandles:
    legobj.set_linewidth(3.0)
    
fig.suptitle('Verteilung des Stromverbrauches für erneuerbare\n sowie nicht erneuerbare Energieträger in Prozent', fontsize=20)

#Move other axis out of the way
pos = ax[0][0].get_position()
pos.y1 = 0.99    #height
ax[0][0].set_position(pos)

plt.tight_layout()
plt.savefig("../Bilder/Verteilung Stromverbrauch Energieträger prozent", bbox_inches = 'tight',pad_inches = 0.08)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [29]:
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ ")
print("stellt die stromprouktion über die Jahre aufgeteilt auf die einzelnen energieträger dar")
print("Bin mir aber ziemlich sicher dass die Power_origin_percent auf verbrauchten strom deuten")
print("Siehe anteil an Nuklear sowie großer Kohleanteil")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ ")


colors = ["orange", "cyan"]



li_ToPLot = ["power_consumption_solar_avg","power_consumption_wind_avg",]

li_Legend = ["Photovoltaik","Wind"]

fig, ax = plt.subplots(figsize=(12, 5))
#labels = dfToPlot.index
df_2015_2019_weekly = df_2015_2019[li_ToPLot].resample("W").mean()
dfToPlot = df_2015_2019_weekly[li_ToPLot]
dfToPlot.plot(kind = "area",ax=ax, color = colors )
leg = ax.legend(loc="upper left",  labels = li_Legend, fontsize = 14,facecolor='#f5f5f5', framealpha=1)

ax.set_xlabel('')
ax.set_ylabel('Stromproduktion [MW]', fontsize = 14)
fig.suptitle('Vergleich von Wind mit Photovoltaik über die Jahre 2015-2020', fontsize=20, x = 0.45)

for legobj in leg.legendHandles:
    legobj.set_linewidth(5.0)


#Move other axis out of the way
pos = ax.get_position()
pos.y1 = 0.95   #height
ax.set_position(pos)
ax.get_yaxis().set_major_formatter(matplotlib.ticker.FuncFormatter(lambda x, loc: locale.format_string('%d', x, 1)))
plt.tight_layout()
plt.savefig("../Bilder/Vergleich_Wid_Solar", bbox_inches = 'tight',pad_inches = 0.08)

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ 
stellt die stromprouktion über die Jahre aufgeteilt auf die einzelnen energieträger dar
Bin mir aber ziemlich sicher dass die Power_origin_percent auf verbrauchten strom deuten
Siehe anteil an Nuklear sowie großer Kohleanteil
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ 


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Primärenergie

In [30]:
PE_NR = 1.02 #Non renewable PE factor
PE_R = 0.61  #Renewable PE factor


for PE_Choose in range(0,12,1):
         #We have to create the Dataframe new every time to prevent a bug where only the first month would be plotted
 
        PE_NR = df_OIB_RL6["fPE,n.ern"].loc[PE_Choose] #Non renewable PE factor
        PE_R = df_OIB_RL6["fPE,ern"].loc[PE_Choose]  #Renewable PE factor

        df_2015_2019["PE_renewable"] = (df_2015_2019["power_consumption_biomass_avg"] + df_2015_2019["power_consumption_hydro_avg"] \
                                + df_2015_2019["power_consumption_solar_avg"] + df_2015_2019["power_consumption_wind_avg"] \
                                + df_2015_2019["power_consumption_geothermal_avg"]) * PE_R

        df_2015_2019["PE_not_renewable"] = (df_2015_2019["power_consumption_coal_avg"] + df_2015_2019["power_consumption_gas_avg"] \
                                + df_2015_2019["power_consumption_nuclear_avg"] + df_2015_2019["power_consumption_oil_avg"] \
                                + df_2015_2019["power_consumption_unknown_avg"]) * PE_NR





df_2015_2019["PE_renewable"] = (df_2015_2019["power_consumption_biomass_avg"] + df_2015_2019["power_consumption_hydro_avg"] \
                            + df_2015_2019["power_consumption_solar_avg"] + df_2015_2019["power_consumption_wind_avg"] \
                            + df_2015_2019["power_consumption_geothermal_avg"]) * PE_R

df_2015_2019["PE_not_renewable"] = (df_2015_2019["power_consumption_coal_avg"] + df_2015_2019["power_consumption_gas_avg"] \
                            + df_2015_2019["power_consumption_nuclear_avg"] + df_2015_2019["power_consumption_oil_avg"] \
                            + df_2015_2019["power_consumption_unknown_avg"]) * PE_NR

df_2015_2019["PE_renewable_percent"] = 100 * df_2015_2019["PE_renewable"] / \
                                            (df_2015_2019["PE_renewable"] + df_2015_2019["PE_not_renewable"])

df_2015_2019["PE_not_renewable_percent"] = 100 * df_2015_2019["PE_not_renewable"] / \
                                            (df_2015_2019["PE_renewable"] + df_2015_2019["PE_not_renewable"])




In [31]:

def makeDF():
    dfToPlot = pd.DataFrame( np.linspace(0,0,len(df_2015_2019)) )
    dfToPlot["datetime"] = df_2015_2019["month"]
    dfToPlot = dfToPlot.drop("datetime", axis = 1)
    dfToPlot["PE_RE_consumption"] = 0
    dfToPlot["PE_NRE_consumption"] = 0
    
    
    li_RE_consumption = [(df_2015_2019["power_consumption_biomass_avg"] \
                                + df_2015_2019["power_consumption_hydro_avg"] \
                                + df_2015_2019["power_consumption_solar_avg"] \
                                + df_2015_2019["power_consumption_wind_avg"] \
                                + df_2015_2019["power_consumption_geothermal_avg"] \
                                + df_2015_2019["power_consumption_hydro_discharge_avg"])]
    
    li_NRE_consumption = [(df_2015_2019["power_consumption_coal_avg"] \
                                    + df_2015_2019["power_consumption_gas_avg"] \
                                    + df_2015_2019["power_consumption_nuclear_avg"] \
                                    + df_2015_2019["power_consumption_oil_avg"] \
                                    + df_2015_2019["power_consumption_unknown_avg"])]
     
    
    
    for row in range(0,len(df_2015_2019),1):
        if df_2015_2019["month"][row] == 1:
            dfToPlot["PE_RE_consumption"].loc[row] = li_RE_consumption[0][row] * df_OIB_RL6["fPE,ern"].loc[0]
            dfToPlot["PE_NRE_consumption"].loc[row] = li_NRE_consumption[0][row] * df_OIB_RL6["fPE,n.ern"].loc[0] 
            
        elif df_2015_2019["month"][row] == 2:
            dfToPlot["PE_RE_consumption"].loc[row] = li_RE_consumption[0][row] * df_OIB_RL6["fPE,ern"].loc[1]
            dfToPlot["PE_NRE_consumption"].loc[row] = li_NRE_consumption[0][row] * df_OIB_RL6["fPE,n.ern"].loc[1]
            
        elif df_2015_2019["month"][row] == 3:  
            dfToPlot["PE_RE_consumption"].loc[row] = li_RE_consumption[0][row] * df_OIB_RL6["fPE,ern"].loc[2]
            dfToPlot["PE_NRE_consumption"].loc[row] = li_NRE_consumption[0][row] * df_OIB_RL6["fPE,n.ern"].loc[2]
            
        elif df_2015_2019["month"][row] == 4:
            dfToPlot["PE_RE_consumption"].loc[row] = li_RE_consumption[0][row] * df_OIB_RL6["fPE,ern"].loc[3]
            dfToPlot["PE_NRE_consumption"].loc[row] = li_NRE_consumption[0][row] * df_OIB_RL6["fPE,n.ern"].loc[3]
            
        elif df_2015_2019["month"][row] == 5:
            dfToPlot["PE_RE_consumption"].loc[row] = li_RE_consumption[0][row] * df_OIB_RL6["fPE,ern"].loc[4]
            dfToPlot["PE_NRE_consumption"].loc[row] = li_NRE_consumption[0][row] * df_OIB_RL6["fPE,n.ern"].loc[4]
            
        elif df_2015_2019["month"][row] == 6:
            dfToPlot["PE_RE_consumption"].loc[row] = li_RE_consumption[0][row] * df_OIB_RL6["fPE,ern"].loc[5]
            dfToPlot["PE_NRE_consumption"].loc[row] = li_NRE_consumption[0][row] * df_OIB_RL6["fPE,n.ern"].loc[5]
            
        elif df_2015_2019["month"][row] == 7:
            dfToPlot["PE_RE_consumption"].loc[row] = li_RE_consumption[0][row] * df_OIB_RL6["fPE,ern"].loc[6]
            dfToPlot["PE_NRE_consumption"].loc[row] = li_NRE_consumption[0][row] * df_OIB_RL6["fPE,n.ern"].loc[6]
            
        elif df_2015_2019["month"][row] == 8:
            dfToPlot["PE_RE_consumption"].loc[row] = li_RE_consumption[0][row] * df_OIB_RL6["fPE,ern"].loc[7]
            dfToPlot["PE_NRE_consumption"].loc[row] = li_NRE_consumption[0][row] * df_OIB_RL6["fPE,n.ern"].loc[7]
            
        elif df_2015_2019["month"][row] == 9:
            dfToPlot["PE_RE_consumption"].loc[row] = li_RE_consumption[0][row] * df_OIB_RL6["fPE,ern"].loc[8]
            dfToPlot["PE_NRE_consumption"].loc[row] = li_NRE_consumption[0][row] * df_OIB_RL6["fPE,n.ern"].loc[8]
            
        elif df_2015_2019["month"][row] == 10:
            dfToPlot["PE_RE_consumption"].loc[row] = li_RE_consumption[0][row] * df_OIB_RL6["fPE,ern"].loc[9]
            dfToPlot["PE_NRE_consumption"].loc[row] = li_NRE_consumption[0][row] * df_OIB_RL6["fPE,n.ern"].loc[9]
            
        elif df_2015_2019["month"][row] == 11:
            dfToPlot["PE_RE_consumption"].loc[row] = li_RE_consumption[0][row] * df_OIB_RL6["fPE,ern"].loc[10]
            dfToPlot["PE_NRE_consumption"].loc[row] = li_NRE_consumption[0][row] * df_OIB_RL6["fPE,n.ern"].loc[10]
            
        elif df_2015_2019["month"][row] == 12:
            dfToPlot["PE_RE_consumption"].loc[row] = li_RE_consumption[0][row] * df_OIB_RL6["fPE,ern"].loc[11]
            dfToPlot["PE_NRE_consumption"].loc[row] = li_NRE_consumption[0][row] * df_OIB_RL6["fPE,n.ern"].loc[11]
    
                               
    

 
    return dfToPlot

In [45]:
fig, ax = plt.subplots(4,2, figsize=(12, 10),gridspec_kw={'width_ratios': [5, 1]})
fig.suptitle('Primärenergie des Jahres 2019', fontsize=20)
tickPos = 0


li_Months = ["Jänner","Februar","März","April","Mai","Juni","Juli","August","September","Oktober","November","Dezember"]
li_Quarter = ["Quartal: Januar - März", "Quartal: April - Juni", "Quartal: Juli - September", "Quartal: Oktober - Dezember"]


def makeDF():
    dfToPlot = pd.DataFrame()
    dfToPlot["PE_RE_consumption"] = 0
    dfToPlot["PE_NRE_consumption"] = 0
    
    dfToPlot["RE_consumption"] = (df_2015_2019["power_consumption_biomass_avg"] \
                                + df_2015_2019["power_consumption_hydro_avg"] \
                                + df_2015_2019["power_consumption_solar_avg"] \
                                + df_2015_2019["power_consumption_wind_avg"] \
                                + df_2015_2019["power_consumption_geothermal_avg"] \
                                + df_2015_2019["power_consumption_hydro_discharge_avg"])
    
    dfToPlot["NRE_consumption"] = (df_2015_2019["power_consumption_coal_avg"] \
                                    + df_2015_2019["power_consumption_gas_avg"] \
                                    + df_2015_2019["power_consumption_nuclear_avg"] \
                                    + df_2015_2019["power_consumption_oil_avg"] \
                                    + df_2015_2019["power_consumption_unknown_avg"]) 
     
    
    
    for row in range(0,len(df_2015_2019),1):
        if df_2015_2019["month"][row] == 1:
            dfToPlot["PE_RE_consumption"][row] = dfToPlot["RE_consumption"][row] * df_OIB_RL6["fPE,ern"].loc[0]
            dfToPlot["PE_NRE_consumption"][row] = dfToPlot["NRE_consumption"][row] * df_OIB_RL6["fPE,n.ern"].loc[0] 
            
        elif df_2015_2019["month"][row] == 2:
            dfToPlot["PE_RE_consumption"][row] = dfToPlot["RE_consumption"][row] * df_OIB_RL6["fPE,ern"].loc[1]
            dfToPlot["PE_NRE_consumption"][row] = dfToPlot["NRE_consumption"][row] * df_OIB_RL6["fPE,n.ern"].loc[1]
            
        elif df_2015_2019["month"][row] == 3:  
            dfToPlot["PE_RE_consumption"][row] = dfToPlot["RE_consumption"][row] * df_OIB_RL6["fPE,ern"].loc[2]
            dfToPlot["PE_NRE_consumption"][row] = dfToPlot["NRE_consumption"][row] * df_OIB_RL6["fPE,n.ern"].loc[2]
            
        elif df_2015_2019["month"][row] == 4:
            dfToPlot["PE_RE_consumption"][row] = dfToPlot["RE_consumption"][row] * df_OIB_RL6["fPE,ern"].loc[3]
            dfToPlot["PE_NRE_consumption"][row] = dfToPlot["NRE_consumption"][row] * df_OIB_RL6["fPE,n.ern"].loc[3]
            
        elif df_2015_2019["month"][row] == 5:
            dfToPlot["PE_RE_consumption"][row] = dfToPlot["RE_consumption"][row] * df_OIB_RL6["fPE,ern"].loc[4]
            dfToPlot["PE_NRE_consumption"][row] = dfToPlot["NRE_consumption"][row] * df_OIB_RL6["fPE,n.ern"].loc[4]
            
        elif df_2015_2019["month"][row] == 6:
            dfToPlot["PE_RE_consumption"][row] = dfToPlot["RE_consumption"][row] * df_OIB_RL6["fPE,ern"].loc[5]
            dfToPlot["PE_NRE_consumption"][row] = dfToPlot["NRE_consumption"][row] * df_OIB_RL6["fPE,n.ern"].loc[5]
            
        elif df_2015_2019["month"][row] == 7:
            dfToPlot["PE_RE_consumption"][row] = dfToPlot["RE_consumption"][row] * df_OIB_RL6["fPE,ern"].loc[6]
            dfToPlot["PE_NRE_consumption"][row] = dfToPlot["NRE_consumption"][row] * df_OIB_RL6["fPE,n.ern"].loc[6]
            
        elif df_2015_2019["month"][row] == 8:
            dfToPlot["PE_RE_consumption"][row] = dfToPlot["RE_consumption"][row] * df_OIB_RL6["fPE,ern"].loc[7]
            dfToPlot["PE_NRE_consumption"][row] = dfToPlot["NRE_consumption"][row] * df_OIB_RL6["fPE,n.ern"].loc[7]
            
        elif df_2015_2019["month"][row] == 9:
            dfToPlot["PE_RE_consumption"][row] = dfToPlot["RE_consumption"][row] * df_OIB_RL6["fPE,ern"].loc[8]
            dfToPlot["PE_NRE_consumption"][row] = dfToPlot["NRE_consumption"][row] * df_OIB_RL6["fPE,n.ern"].loc[8]
            
        elif df_2015_2019["month"][row] == 10:
            dfToPlot["PE_RE_consumption"][row] = dfToPlot["RE_consumption"][row] * df_OIB_RL6["fPE,ern"].loc[9]
            dfToPlot["PE_NRE_consumption"][row] = dfToPlot["NRE_consumption"][row] * df_OIB_RL6["fPE,n.ern"].loc[9]
            
        elif df_2015_2019["month"][row] == 11:
            dfToPlot["PE_RE_consumption"][row] = dfToPlot["RE_consumption"][row] * df_OIB_RL6["fPE,ern"].loc[10]
            dfToPlot["PE_NRE_consumption"][row] = dfToPlot["NRE_consumption"][row] * df_OIB_RL6["fPE,n.ern"].loc[10]
            
        elif df_2015_2019["month"][row] == 12:
            dfToPlot["PE_RE_consumption"][row] = dfToPlot["RE_consumption"][row] * df_OIB_RL6["fPE,ern"].loc[11]
            dfToPlot["PE_NRE_consumption"][row] = dfToPlot["NRE_consumption"][row] * df_OIB_RL6["fPE,n.ern"].loc[11]

    return dfToPlot


for quarter in range(0,4,1):
    df_ToPlot = makeDF()
    #Filter the DataFrame to all Values in the Year 2019
    df_ToPlot = df_ToPlot[df_ToPlot.index.year == 2019]
    #Filter the DataFrame to all Values for the selected month
    #The Xticklabels wont work properly if the next line is present
    df_ToPlot = df_ToPlot[df_ToPlot.index.quarter == (quarter+1)]
    #For some reason we have to create series to transfer the new data to a new dataframe or else 
    #we cant delete xticklabels
    dfToPlot_BugFree = pd.DataFrame()    
    ts = pd.Series(df_ToPlot['PE_RE_consumption'].values)
    dfToPlot_BugFree["PE_RE_consumption"] = ts
    ts = pd.Series(df_ToPlot['PE_NRE_consumption'].values)
    dfToPlot_BugFree["PE_NRE_consumption"] = ts 
    
    dfToPlot_BugFree.plot(kind="area", color=["green","grey"], stacked=True, ax=ax[quarter][0], legend=False);
    
    #ax2 = ax[quarter].twinx()  # instantiate a second axes that shares the same x-axis
    #sns.lineplot(x ="hour", y= "PE_renewable_percent" , data = df_2015_2019 , ax = ax2, color = "yellow")
    #dfToPlot["PE_renewable"].plot(kind = "line" ,ax = ax2 )
    
    ticklen = len(dfToPlot_BugFree)/3
    ticklen = [0,ticklen,ticklen*2]
    
    ax[quarter][0].set_xticks(ticklen)
    ax[quarter][0].set_xticklabels([li_Months[tickPos],li_Months[tickPos+1],li_Months[tickPos+2]], fontsize = 14)
    tickPos += 3
    ax[quarter][0].tick_params(axis="x", which='both',length=0)
    ax[quarter][0].set_ylabel("Primärenergie\n[MW]", fontsize = 14)
    ax[quarter][0].set_xlabel("")
    ax[quarter][0].set_title(li_Quarter[quarter])
    ax[quarter][0].set_ylim(0,10000)
    ax[quarter][0].get_yaxis().set_major_formatter(matplotlib.ticker.FuncFormatter(lambda x, loc: locale.format_string('%d', x, 1)))
    dti_2030 = pd.date_range('2030-01-01', periods=len(dfToPlot_BugFree), freq='H')
    dfToPlot_BugFree = dfToPlot_BugFree.set_index(dti_2030)
    dfToPlot_BugFree["PE_RE_consumption"] = 100 * dfToPlot_BugFree["PE_RE_consumption"] / \
                                            (dfToPlot_BugFree["PE_NRE_consumption"]+ dfToPlot_BugFree["PE_RE_consumption"])
    dfToPlot_BugFree["PE_NRE_consumption"] = 100- dfToPlot_BugFree["PE_RE_consumption"]
    dfToPlot_BugFree.resample("Y").mean().plot(kind='bar', stacked=True, ax = ax[quarter][1], color = ["green","grey"],legend=False)
    ax[quarter][1].set_xlabel("")
    ax[quarter][1].set_ylabel("Prozent [%]")
    ax[quarter][1].set_xticklabels("")
    ax[quarter][1].tick_params(
    axis='x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False) # labels along the bottom edge are off


leg = fig.legend(labels = ["Erneuerbar","Nicht Erneuerbar"], bbox_to_anchor = [0.95, 1],
             prop={'size': 14},facecolor='#f5f5f5', framealpha=1)
#change width of legend color lines
for legobj in leg.legendHandles:
    legobj.set_linewidth(10.0)
    

#Move other axis out of the way
pos = ax[0][0].get_position()
pos.y1 = 0.96    #height
ax[0][0].set_position(pos)

plt.tight_layout()
plt.savefig("../Bilder/PE bedarf 2019 nach Quartale", bbox_inches = 'tight',pad_inches = 0.08)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [33]:
dfToPlot_BugFree

,PE_RE_consumption,PE_NRE_consumption
2030-01-01 00:00:00,34.332927,65.667073
2030-01-01 01:00:00,30.990672,69.009328
2030-01-01 02:00:00,28.474860,71.525140
2030-01-01 03:00:00,26.081500,73.918500
2030-01-01 04:00:00,33.166955,66.833045
...,...,...
2030-04-02 19:00:00,40.143291,59.856709
2030-04-02 20:00:00,38.628267,61.371733
2030-04-02 21:00:00,38.554958,61.445042
2030-04-02 22:00:00,37.911941,62.088059


In [34]:
fig, ax = plt.subplots( figsize=(5, 6))
#dfToPlot_BugFree = dfToPlot_BugFree
dfToPlot_BugFree["PE_RE_consumption"] = 100 * dfToPlot_BugFree["PE_RE_consumption"] / \
                                            (dfToPlot_BugFree["PE_NRE_consumption"]+ dfToPlot_BugFree["PE_RE_consumption"])
dfToPlot_BugFree["PE_NRE_consumption"] = 100- dfToPlot_BugFree["PE_RE_consumption"]
dfToPlot_BugFree.resample("Y").mean().plot(kind='bar', stacked=True, ax = ax, color = ["green","grey"],legend=False)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [35]:
dfToPlot

,power_consumption_solar_avg,power_consumption_wind_avg
datetime,,
2015-01-04 00:00:00+00:00,22.921603,911.359954
2015-01-11 00:00:00+00:00,30.600123,1066.657554
2015-01-18 00:00:00+00:00,43.313726,698.431425
2015-01-25 00:00:00+00:00,25.444136,544.489834
2015-02-01 00:00:00+00:00,37.871861,557.720102
...,...,...
2019-12-08 00:00:00+00:00,71.469154,1428.122481
2019-12-15 00:00:00+00:00,61.445153,1498.229860
2019-12-22 00:00:00+00:00,74.065936,1374.106933


In [46]:
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ ")
print("Stellt die Aufteilung des Primärenergiebedarfs zwischen erneuerbar und nicht erneuerbar dar ")
print("Ausgewählt für berechnung des Primärenergiebedarfs ist der verbrauchte Strom in Österreich")
print("Anzumerken ist der Einbruch im Jahr 2016 (Höchstwahrscheinlich fehlerhafte Daten und kein plötzlicher Einbruch)")
print("Ansonsten ist schön zu sehen das die Wintermonate mit nicht erneuerbaren mitteln gedeckt werden müssen")
print("TO-DO: Weiteres Diagramm dass das Verhältnis zwischen RE und NE über die einzelnen Jahre darstellt(denke histogramm)")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ ")

PE_NR = 1.02 #Non renewable PE factor
PE_R = 0.61  #Renewable PE factor

df_2015_2019["PE_renewable"] = (df_2015_2019["power_consumption_biomass_avg"] + df_2015_2019["power_consumption_hydro_avg"] \
                            + df_2015_2019["power_consumption_solar_avg"] + df_2015_2019["power_consumption_wind_avg"] \
                            + df_2015_2019["power_consumption_geothermal_avg"] + df_2015_2019["power_consumption_hydro_discharge_avg"]) * PE_R

df_2015_2019["PE_not_renewable"] = (df_2015_2019["power_consumption_coal_avg"] + df_2015_2019["power_consumption_gas_avg"] \
                            + df_2015_2019["power_consumption_nuclear_avg"] + df_2015_2019["power_consumption_oil_avg"] \
                            + df_2015_2019["power_consumption_unknown_avg"]) * PE_NR

df_2015_2019["PE_renewable_percent"] = 100 * df_2015_2019["PE_renewable"] / \
                                            (df_2015_2019["PE_renewable"] + df_2015_2019["PE_not_renewable"])

df_2015_2019["PE_not_renewable_percent"] = 100 * df_2015_2019["PE_not_renewable"] / \
                                            (df_2015_2019["PE_renewable"] + df_2015_2019["PE_not_renewable"])

fig, ax = plt.subplots(2,2, figsize=(12, 10))
plt.rcParams.update({'font.size': 13})
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#Middle left: Primary Energy Monthly stacked with area
dfToPlot = pd.DataFrame()
dfToPlot["PE_renewable"] = df_2015_2019["PE_renewable"].resample("M").mean()
dfToPlot["PE_not_renewable"] = df_2015_2019["PE_not_renewable"].resample("M").mean()

dfToPlot.plot(kind="area", color=["green","grey"], stacked=True, ax=ax[0][0], legend=False);

ax[0][0].set_title("Monatlich")
ax[0][0].set_xlabel('')
ax[0][0].set_ylabel('Primärenergie [MW]', fontsize=14)
ax[0][0].get_yaxis().set_major_formatter(matplotlib.ticker.FuncFormatter(lambda x, loc: locale.format_string('%d', x, 1)))
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#Middle right: Primary Energy yearly stacked with bars
dfToPlot = pd.DataFrame()
dfToPlot["PE_renewable"] = df_2015_2019["PE_renewable"].resample("Y").mean()
dfToPlot["PE_not_renewable"] = df_2015_2019["PE_not_renewable"].resample("Y").mean()

dfToPlot.plot(kind='bar', stacked=True, ax = ax[0][1], color = ["green","grey"],legend=False)

ax[0][1].set_title("Jährlich")
ax[0][1].set_xlabel('')
ax[0][1].set_ylabel('Primärenergie [MW]', fontsize=14)
ax[0][1].set_xticklabels(["2015","2016","2017","2018","2019"])

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#Bottom left: Primary Energy weekly resampled in percent
dfToPlot = pd.DataFrame()
dfToPlot["PE_renewable_percent"] = df_2015_2019["PE_renewable_percent"].resample("M").mean()
dfToPlot["PE_not_renewable_percent"] = df_2015_2019["PE_not_renewable_percent"].resample("M").mean()

dfToPlot.plot(kind="area", color=["green","grey"], stacked=True, ax=ax[1][0], legend=False);

ax[1][0].set_title("Monatlich")
ax[1][0].set_xlabel('')
ax[1][0].set_ylabel('Primärenergie [%]', fontsize=14)
ax[1][0].get_yaxis().set_major_formatter(matplotlib.ticker.FuncFormatter(lambda x, loc: locale.format_string('%d', x, 1)))
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#Bottom right: Primary Energy yearly resampled in percent
dfToPlot = pd.DataFrame()
dfToPlot["PE_renewable_percent"] = df_2015_2019["PE_renewable_percent"].resample("Y").mean()
dfToPlot["PE_not_renewable_percent"] = df_2015_2019["PE_not_renewable_percent"].resample("Y").mean()

dfToPlot.plot(kind='bar', stacked=True, ax = ax[1][1], color = ["green","grey"],legend=False)

ax[1][1].set_title("Jährlich")
ax[1][1].set_xlabel('')
ax[1][1].set_ylabel('Primärenergie [%]', fontsize=14)
ax[1][1].set_xticklabels(["2015","2016","2017","2018","2019"])
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
fig.suptitle('Primärenergie der Jahre 2015-2020', fontsize=20)
#create a legend for the whole figure
leg = fig.legend( labels = ["PE Erneuerbar","PE nicht Erneuerbar"], bbox_to_anchor = [0.63, 0.935],
             prop={'size': 14},facecolor='#f5f5f5', framealpha=1)
#change width of legend color lines

for i in range(0,2,1):
    for j in range(0,2,1):
        ax[j][i].yaxis.set_tick_params(labelsize=14)
        ax[j][i].xaxis.set_tick_params(labelsize=14)


for legobj in leg.legendHandles:
    legobj.set_linewidth(10.0)
#Move other axis out of the way
pos = ax[0][0].get_position()
pos.y0 = 0.95    #height
ax[0][0].set_position(pos)
plt.tight_layout()
plt.savefig("../Bilder/PE bedarf Täglich-wöchentlich-monatlich", bbox_inches = 'tight',pad_inches = 0.08)

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ 
Stellt die Aufteilung des Primärenergiebedarfs zwischen erneuerbar und nicht erneuerbar dar 
Ausgewählt für berechnung des Primärenergiebedarfs ist der verbrauchte Strom in Österreich
Anzumerken ist der Einbruch im Jahr 2016 (Höchstwahrscheinlich fehlerhafte Daten und kein plötzlicher Einbruch)
Ansonsten ist schön zu sehen das die Wintermonate mit nicht erneuerbaren mitteln gedeckt werden müssen
TO-DO: Weiteres Diagramm dass das Verhältnis zwischen RE und NE über die einzelnen Jahre darstellt(denke histogramm)
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ 


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [47]:

dfToPlot = pd.DataFrame()

fig, ax = plt.subplots(3,2, figsize=(12, 9))

li_toPlot = ["PE_renewable","PE_not_renewable","PE_renewable_percent","PE_not_renewable_percent",
             "month","hour","quarter","week","date"]




#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#Upper left: Primary Energy for one average day of the year 
dfToPlot = df_2015_2019[li_toPlot].groupby(pd.Grouper(freq='H')).mean()
dfToPlot = dfToPlot[dfToPlot.index.quarter == 4]
dfToPlot["control"] = dfToPlot["PE_renewable"] + dfToPlot["PE_not_renewable"]
dfToPlot["control_Percent"] = dfToPlot["PE_renewable_percent"] + dfToPlot["PE_not_renewable_percent"]

sns.barplot(x="hour", y="control", data=dfToPlot , ax = ax[0][0], color = "grey", ci = 0)
sns.barplot(x="hour", y="PE_renewable", data=dfToPlot , ax = ax[0][0], color = "green", ci = 0)

ax[0][0].set_title("Durchschnittlicher Wintertag", loc = "center" )
ax[0][0].set_xticklabels(["0","","","","4","","","","8","","","","12","","","","16","","","","20","","","","24"]) 
ax[0][0].set_xlabel('')
ax[0][0].set_ylabel('Primärenergie\n[MW]')
ax[0][0].get_yaxis().set_major_formatter(matplotlib.ticker.FuncFormatter(lambda x, loc: locale.format_string('%d', x, 1)))
#~~~~~~~~~~~~~~~~~~     
#Upper Right: Primary Energy for one average day of the year in percent
sns.barplot(x="hour", y="control_Percent", data=dfToPlot , ax = ax[0][1], color = "grey", ci = 0)
sns.barplot(x="hour", y="PE_renewable_percent", data=dfToPlot , ax = ax[0][1], color = "green", ci = 0)
ax[0][1].set_xticklabels(["0","","","","4","","","","8","","","","12","","","","16","","","","20","","","","24"])  
ax[0][1].set_xlabel('')
ax[0][1].set_ylabel('Primärenergie[%]')
ax[0][1].set_yticks([0,25,50,75,100]) 
ax[0][1].set_yticklabels(['0',"25","50","75","100"]) 
#test = df_2015_2019["PE_renewable"].loc[df_2015_2019.index.month == 1].mean()
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#Upper left: Primary Energy for one average day of the year 
dfToPlot = df_2015_2019[li_toPlot].groupby(pd.Grouper(freq='H')).mean()
dfToPlot = dfToPlot[dfToPlot.index.quarter == 2]
dfToPlot["control"] = dfToPlot["PE_renewable"] + dfToPlot["PE_not_renewable"]
dfToPlot["control_Percent"] = dfToPlot["PE_renewable_percent"] + dfToPlot["PE_not_renewable_percent"]

sns.barplot(x="hour", y="control", data=dfToPlot , ax = ax[1][0], color = "grey", ci = 0)
sns.barplot(x="hour", y="PE_renewable", data=dfToPlot , ax = ax[1][0], color = "green", ci = 0)

ax[1][0].set_title("Durchschnittlicher Sommertag", loc = "center" )
ax[1][0].set_xticklabels(["0","","","","4","","","","8","","","","12","","","","16","","","","20","","","","24"]) 
ax[1][0].set_xlabel('')
ax[1][0].set_ylabel('Primärenergie\n[MW]')
ax[1][0].get_yaxis().set_major_formatter(matplotlib.ticker.FuncFormatter(lambda x, loc: locale.format_string('%d', x, 1)))
#~~~~~~~~~~~~~~~~~~     
#Upper Right: Primary Energy for one average day of the year in percent
sns.barplot(x="hour", y="control_Percent", data=dfToPlot , ax = ax[1][1], color = "grey", ci = 0)
sns.barplot(x="hour", y="PE_renewable_percent", data=dfToPlot , ax = ax[1][1], color = "green", ci = 0)
ax[1][1].set_xticklabels(["0","","","","4","","","","8","","","","12","","","","16","","","","20","","","","24"])  
ax[1][1].set_xlabel('')
ax[1][1].set_ylabel('Primärenergie[%]')
ax[1][1].set_yticks([0,25,50,75,100]) 
ax[1][1].set_yticklabels(['0',"25","50","75","100"]) 
 #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~     
 #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~     
#Bottom left: Primary Energy for one average month of the year 
dfToPlot = df_2015_2019[li_toPlot].groupby(pd.Grouper(freq='M')).mean()
dfToPlot["control"] = dfToPlot["PE_renewable"] + dfToPlot["PE_not_renewable"]
dfToPlot["control_Percent"] = dfToPlot["PE_renewable_percent"] + dfToPlot["PE_not_renewable_percent"]

sns.barplot(x="month", y="control", data=dfToPlot , ax = ax[2][0], color = "grey", ci = 0)
sns.barplot(x="month", y="PE_renewable", data=dfToPlot , ax = ax[2][0], color = "green", ci = 0)

ax[2][0].set_title("Durchschnittlicher Monat", loc = "center" )
ax[2][0].set_xticklabels(["Jan","Feb","Mär","Apr","Mai","Jun","Jul",
                       "Aug","Sep","Okt","Nov","Dez"])
ax[2][0].set_xlabel('')
ax[2][0].set_ylabel('Primärenergie\n[MW)')
ax[2][0].get_yaxis().set_major_formatter(matplotlib.ticker.FuncFormatter(lambda x, loc: locale.format_string('%d', x, 1)))
#~~~~~~~~~~~~~~~~~~  
#Bottom right: Primary Energy for one average month of the year in percent
sns.barplot(x="month", y="control_Percent", data=dfToPlot , ax = ax[2][1], color = "grey", ci = 0)
sns.barplot(x="month", y="PE_renewable_percent", data=dfToPlot , ax = ax[2][1], color = "green", ci = 0)

ax[2][1].set_xticklabels(["Jan","Feb","Mär","Apr","Mai","Jun","Jul",
                       "Aug","Sep","Okt","Nov","Dez"]) 
ax[2][1].set_xlabel('')
ax[2][1].set_ylabel('Primärenergie[%]')    
ax[2][1].set_yticks([0,25,50,75,100]) 
ax[2][1].set_yticklabels(['0',"25","50","75","100"])              
    
fig.suptitle('Primärenergie aufgeteilt nach durchschnittlichen Monat bzw. Tag im Sommer/Winter', fontsize=20)
#create a legend for the whole figure
leg = fig.legend(labels = ["Erneuerbar","Nicht Erneuerbar"], bbox_to_anchor = [0.63, 0.935],
             prop={'size': 14},facecolor='#f5f5f5', framealpha=1)
#change width of legend color lines
for legobj in leg.legendHandles:
    legobj.set_linewidth(10.0)
#For whatever reason we have to allocate the colors of the legend manually, otherwise the colors would be both black    
leg.legendHandles[0].set_color('green')
leg.legendHandles[1].set_color('grey')    
#Move other axis out of the way
pos = ax[0][0].get_position()
pos.y0 = 1    #height
ax[0][0].set_position(pos)
plt.tight_layout()
plt.savefig("../Bilder/PE bedarf Durchschnittlichertag", bbox_inches = 'tight',pad_inches = 0.08)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# CO2 Emissionen

In [48]:
fig, ax = plt.subplots(4, 2, figsize=(12,12),gridspec_kw={'width_ratios': [3, 1]})


li_Seasons = ["Frühling", "Sommer", "Herbst", "Winter"]
for i in range(0,4,1):
    
    sns.lineplot(x="hour", y="carbon_intensity_avg"
                ,hue="year", data=df_2015_2019[df_2015_2019.index.quarter == i+1] , ax = ax[i][0] 
             , palette=["C0", "C1", "C2","C3", "C4"], legend = False , ci = 0)

    ax[i][1].bar(x =  ["2015","2016","2017","2018","2019"],
          height = df_2015_2019[df_2015_2019.index.quarter == i+1]["carbon_intensity_avg"].resample("Y").mean() ,
                color=["C0", "C1", "C2","C3", "C4"])
    
    
    
    ax[i][0].set_xlabel('')
    ax[i][0].set_ylabel('CO$_2$-Intensität\n [gCO$_2$eq/kWh]')
    ax[i][0].set_title('Durchschnittlicher Tag im ' + li_Seasons[i])
    ax[i][0].set_ylim(100,400)
    
    ax[i][1].set_ylabel('CO$_2$-Intensität\n [gCO$_2$eq/kWh]') 
    ax[i][1].set_title('Durchschnittliches Quartal\n im ' + li_Seasons[i])
    ax[i][1].set_xticklabels(["15","16","17","18","19"])
    ax[i][1].set_ylim(0,350)
    
fig.suptitle('Durchschnittliche tägliche CO$_2$ Emissionen über die Jahre 2015-2020', fontsize=20)
leg = fig.legend(labels = ["2015","2016","2017","2018","2019"], 
                 bbox_to_anchor = [0.78, 0.96],prop={'size': 14}, ncol = 6,facecolor='#f5f5f5', framealpha=1)

#change width of legend color lines
for legobj in leg.legendHandles:
    legobj.set_linewidth(5.0)

    
#Move other axis out of the way
pos = ax[0][0].get_position()
pos.y1 = 0.97    #height
ax[0][0].set_position(pos)
plt.tight_layout()
plt.savefig("../Bilder/CO2 Emissionen über einen durchschnittlichen Tag pro Jahr", bbox_inches = 'tight',pad_inches = 0.08)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [53]:
fig, ax = plt.subplots(5,2, figsize=(12,12),constrained_layout=True)




li_Months = ["","Feb","Apr","Jun","Aug","Okt","Dez"]

for j,year in enumerate([2015,2016,2017,2018,2019]):
    sns.lineplot(x="month", y="carbon_intensity_avg"
                , data=df_2015_2019.loc[df_2015_2019.index.year==year].resample("D").mean() 
                 , ax = ax[j][0] , color = "grey", legend = False)
    
    ax[j][0].set_xticklabels(li_Months)
    ax[j][0].set_ylabel("CO$_2$-Intensität \n[gCO$_2$eq/kWh]")
    ax[j][0].set_xlabel("")
    ax[j][0].set_title("Jahr " + str(year))
    i = 0
    while i != 11:
        #draw horizontal line
        ax[j][0].plot([i+1, i+2], [df_OIB_RL6["fCO2eq"][i],df_OIB_RL6["fCO2eq"][i]], 
                 linewidth=3, color = "red")
        if i+1 !=11:
            #draw vertical line
            ax[j][0].plot([i+2, i+2], [df_OIB_RL6["fCO2eq"][i+1],df_OIB_RL6["fCO2eq"][i]],
                 linewidth=3 ,color = "red")   
        i += 1
        
    #invisible point for label
    ax[j][0].plot([1, 1], [1,1], 
                 linewidth=3, color = "red" , label = "Richtwert OIB_RL6(2018)")    
   
  

for j,year in enumerate([2018,2019],3):
    
    temp = np.mean(OE_NORM,axis=1)
    df_ToPlot = pd.DataFrame()
    
    s = df_2018.loc[df_2018.index.year==year].resample("M").mean().subtract(273.15)
    df_ToPlot["Isttemperatur"] = s["latest_forecasted_temperature_avg"]
    df_ToPlot["month"] = s["month"]
    df_ToPlot["Normtemperatur"] = temp

    
    sns.lineplot(x = "month", y="Isttemperatur", data=df_ToPlot
                 , ax = ax[j][1] , palette=["C0"]  , legend = False, color = "blue")
    sns.lineplot(x = "month", y="Normtemperatur", data=df_ToPlot
                 , ax = ax[j][1] , palette=["C0"]  , legend = False, color = "green")

    
    ax[j][1].set_xticklabels(li_Months)
    ax[j][1].set_ylabel("Außentemperatur [°C]")
    ax[j][1].set_xlabel("")
    ax[j][1].set_title("gemittelte Außentemperatur")
    
fig.suptitle('CO$_2$ Emissionen verglichen mit der OIB-RL6', fontsize=20)
leg = fig.legend(labels = ["CO$_2$ Emissionen","Richtwert OIB_RL6(2018)","Temperatur","Normtemperatur"], 
                 bbox_to_anchor = [0.89, 0.96],prop={'size': 14}, ncol = 4,facecolor='#f5f5f5', framealpha=1)

#change width of legend color lines
for legobj in leg.legendHandles:
    legobj.set_linewidth(5.0)
    
li_Color = ["grey","red","blue","green"]
for i in range(0,4,1):
    leg.legendHandles[i].set_color(li_Color[i]) 

box1 = ax[4][0].get_position()
box2 = ax[0][0].get_position()
box3 = ax[4][1].get_position()

for i in range(0,3,1):
    ax[i][1].remove()
    box2 = ax[i][0].get_position()
    ax[i][0].set_position([box2.x0, box2.y0,box3.x1-box1.x0, box1.y1-box1.y0])


pos = ax[1][0].get_position()
pos.y0 -= 0.02
pos.y1 -= 0.02
ax[1][0].set_position(pos)

pos = ax[2][0].get_position()
pos.y0 -= 0.04
pos.y1 -= 0.04
ax[2][0].set_position(pos)


#Move other axis out of the way
for row in range(3,5,1):
    for col in range(0,2,1):
        pos = ax[row][col].get_position()
        pos.y0 -= 0.02 
        pos.y1 -= 0.02
        ax[row][col].set_position(pos)

pos = ax[3][0].get_position()
pos.y0 -= 0.05
pos.y1 -= 0.05
ax[3][0].set_position(pos)

pos = ax[3][1].get_position()
pos.y0 -= 0.05
pos.y1 -= 0.05
ax[3][1].set_position(pos)

pos = ax[4][0].get_position()
pos.y0 -= 0.08
pos.y1 -= 0.08
ax[4][0].set_position(pos)

pos = ax[4][1].get_position()
pos.y0 -= 0.08
pos.y1 -= 0.08
ax[4][1].set_position(pos)

#plt.tight_layout()
plt.savefig("../Bilder/CO2 verglichen mit OIB6", bbox_inches = 'tight',pad_inches = 0.08)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [50]:
fig, ax = plt.subplots(4,3, figsize=(12, 10))

fig.suptitle('CO$_2$ Emissionen des Jahres 2018', fontsize=20)

li_Months = ["Jänner","Februar","März","April","Mai","Juni","Juli","August","September","Oktober","November","Dezember"]
i = -1
for row in range(0,4,1):   
    for col in range(0,3,1):
        i += 1
        dfToPlot = pd.DataFrame()
    
        dfToPlot = df_2015_2019[df_2015_2019.index.year == 2018]
        #Filter the DataFrame to all Values for the selected month
        #The Xticklabels wont work properly if the next line is present
        dfToPlot = dfToPlot[dfToPlot.index.month == (i+1)]
        
        #Draw horizontal line
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        #Draw red limitline
        length = len(dfToPlot)/24
        ax[row][col].plot([0, len(dfToPlot)/24], [df_OIB_RL6["fCO2eq"][i],df_OIB_RL6["fCO2eq"][i]], 
                     linewidth=3, color = "red")
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        #Draw Co2 intensity
        sns.lineplot(x="date", y="carbon_intensity_avg"
                    , data=dfToPlot 
                     , ax = ax[row][col] , color = "grey" , legend = False )
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        #Draw weekly Co2 average
        dfToPlot = dfToPlot[dfToPlot.index.month == (i+1)].resample("W").mean()

        plot_av = dfToPlot["carbon_intensity_avg"]

        ax[row][col].plot([0, length], [plot_av[0],plot_av[3]], 
                     linewidth=3, color = "black")
        ax[row][col].set_xlabel("")
        ax[row][col].set_title(li_Months[i])
        ax[row][col].set_ylabel("CO$_2$-Intensität \n[gCO$_2$eq/kWh]")
        ax[row][col].set_ylim(0,350)
        
        
leg = fig.legend(labels = ["OIB-RL6(2018)","CO$_2$ Emissionen","Wöchentliche Trendlinie"], bbox_to_anchor = [0.83, 0.95],
             prop={'size': 14}, ncol = 3,facecolor='#f5f5f5', framealpha=1)
#change width of legend color lines
for legobj in leg.legendHandles:
    legobj.set_linewidth(5.0)
    

#Move other axis out of the way
pos = ax[0][0].get_position()
pos.y0 = 0.97    #height
ax[0][0].set_position(pos)
plt.tight_layout()
plt.savefig("../Bilder/CO2 verglichen mit OIB6 Wochenbasis", bbox_inches = 'tight',pad_inches = 0.08)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [58]:
def ZipObjects(list1, list2):
    difference = []
    zip_object = zip(list1, list2)
    for list1_i, list2_i in zip_object:
        difference.append(list2_i-list1_i)
    return difference


df_Change = pd.DataFrame()
df_Change = df_2015_2019[df_2015_2019.index.year == 2015]
li_2015 = df_Change["carbon_intensity_avg"].resample("W").mean()

df_Change = df_2015_2019[df_2015_2019.index.year == 2016]
li_2016 = df_Change["carbon_intensity_avg"].resample("W").mean()

df_Change = df_2015_2019[df_2015_2019.index.year == 2017]
li_2017 = df_Change["carbon_intensity_avg"].resample("W").mean()

df_Change = df_2015_2019[df_2015_2019.index.year == 2018]
li_2018 = df_Change["carbon_intensity_avg"].resample("W").mean()

df_Change = df_2015_2019[df_2015_2019.index.year == 2019]
li_2019 = df_Change["carbon_intensity_avg"].resample("W").mean()


df_ToPlot = pd.DataFrame()
df_ToPlot["Änderung_2015_2016"] = ZipObjects(li_2015,li_2016) 
df_ToPlot["Änderung_2016_2017"] = ZipObjects(li_2016,li_2017)
df_ToPlot["Änderung_2017_2018"] = ZipObjects(li_2017,li_2018)
df_ToPlot["Änderung_2018_2019"] = ZipObjects(li_2018,li_2019)
 
    
    
df_ToPlot_Pos = df_ToPlot.clip(0, 15000)
df_ToPlot_Neg = df_ToPlot.clip(0, -15000)

fig, ax = plt.subplots( figsize=(14,10))
df_ToPlot_Pos.plot(kind = "area",ax=ax, color = ["#ff3636","#42f572","#f58742","#b3b1b1"], legend = False, linewidth = 0)
df_ToPlot_Neg.plot(kind = "area",ax=ax, color = ["#ff3636","#42f572","#f58742","#b3b1b1"], legend = False, linewidth = 0)

ax.set_ylabel('Änderung der CO$_2$ Emissionen \n[gCO$_2$eq/kWh]', fontsize = 14)
#Set xlabels so we can have words instead of numbers for easy understanding
ax.set_xticks([0,4.3,8.6,12.9,17.3,21.6,25.9,30.3,34.6,38.9,43.3,47.6])
ax.set_xticklabels(["Jan","Feb","Mär","Apr","Mai","Jun","Jul",
                       "Aug","Sep","Okt","Nov","Dez"], fontsize = 14)
ax.set_ylim(-300,200)
fig.suptitle('Änderung der CO$_2$ Emissionen der Jahre 2015-2020', fontsize=20)

leg = fig.legend(labels = ["Änderung 2015-2016)","Änderung 2016-2017","Änderung 2017-2018","Änderung 2018-2019"]
                 , bbox_to_anchor = [0.935, 0.945],prop={'size': 14}, ncol = 4)
for legobj in leg.legendHandles:
    legobj.set_linewidth(5.0)

 

plt.savefig("../Bilder/Änderung der CO2 Intensität",bbox_to_anchor = "tight",pad_inches = 0)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

d:\Users\Marius\anaconda3\lib\site-packages\pandas\plotting\_matplotlib\core.py:1282: UserWarning: Attempting to set identical bottom == top == 0.0 results in singular transformations; automatically expanding.
  ax.set_ylim(None, 0)


In [55]:
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ ")
print("stellt die stromprouktion über die Jahre aufgeteilt auf die einzelnen energieträger dar")
print("Bin mir aber ziemlich sicher dass die Power_origin_percent auf verbrauchten strom deuten")
print("Siehe anteil an Nuklear sowie großer Kohleanteil")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ ")


colors = ["green", "black","purple", "#0088ff", "#b4b88a", "brown", "orange", "cyan","grey","#0000ff","red"]


li_ToPLot = ["carbon_production_biomass_avg","carbon_production_coal_avg",
            "carbon_production_gas_avg","carbon_production_hydro_avg",
            "carbon_production_nuclear_avg","carbon_production_oil_avg",
            "carbon_production_solar_avg","carbon_production_wind_avg",
            "carbon_production_geothermal_avg","carbon_production_hydro_discharge_avg",
             "carbon_production_unknown_avg",]

li_Legend = ["Biomasse","Kohle","Gas","Wasser","Nuklear","Öl","Photovoltaik",
             "Wind","Geothermie","Speicherwasser","Unbekannt"]

fig, ax = plt.subplots(2,figsize=(12, 8))
#labels = dfToPlot.index
dfToPlot = df_2018[li_ToPLot].resample("W").mean()
dfToPlot.plot(kind = "area",ax=ax[0], color = colors )
leg = ax[0].legend(loc="upper left", bbox_to_anchor=(1.025,1.05), title = "Art der Erzeugung", 
                   labels = li_Legend,fontsize = 14,facecolor='#f5f5f5', framealpha=1)

ax[0].set_xlabel('')
ax[0].set_ylabel('CO$_2$ Intensität [gCO$_2$eq/kWh]', fontsize = 14)

li_ToPLot = ["carbon_origin_percent_biomass_avg","carbon_origin_percent_coal_avg",
            "carbon_origin_percent_gas_avg","carbon_origin_percent_hydro_avg",
            "carbon_origin_percent_nuclear_avg","carbon_origin_percent_oil_avg",
            "carbon_origin_percent_solar_avg","carbon_origin_percent_wind_avg",
            "carbon_origin_percent_geothermal_avg","carbon_origin_percent_hydro_discharge_avg",
             "carbon_origin_percent_unknown_avg",]


dfToPlot = df_2018[li_ToPLot].resample("W").mean()
dfToPlot.plot(kind = "area",ax=ax[1], color = colors, legend = False )

ax[1].set_xlabel('')
ax[1].set_ylabel('CO$_2$ Intensität [%]', fontsize = 14)


fig.suptitle('CO$_2$-Mix der Jahre 2018-2020', fontsize=20, x = 0.45)

for legobj in leg.legendHandles:
    legobj.set_linewidth(5.0)


#Move other axis out of the way
pos = ax[0].get_position()
pos.y1 = 0.95   #height
ax[0].set_position(pos)

plt.tight_layout()
plt.savefig("../Bilder/CO2-mix_Absolut_prozent", bbox_inches = 'tight',pad_inches = 0.08)


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ 
stellt die stromprouktion über die Jahre aufgeteilt auf die einzelnen energieträger dar
Bin mir aber ziemlich sicher dass die Power_origin_percent auf verbrauchten strom deuten
Siehe anteil an Nuklear sowie großer Kohleanteil
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ 


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [43]:
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ ")
print("stellt die stromprouktion über die Jahre aufgeteilt auf die einzelnen energieträger dar")
print("Bin mir aber ziemlich sicher dass die Power_origin_percent auf verbrauchten strom deuten")
print("Siehe anteil an Nuklear sowie großer Kohleanteil")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ ")


colors = ["green", "black","purple", "#0088ff", "#b4b88a", "brown", "orange", "cyan","grey","#0000ff","red"]


li_ToPLot = ["carbon_production_biomass_avg","carbon_production_coal_avg",
            "carbon_production_gas_avg","carbon_production_hydro_avg",
            "carbon_production_nuclear_avg","carbon_production_oil_avg",
            "carbon_production_solar_avg","carbon_production_wind_avg",
            "carbon_production_geothermal_avg","carbon_production_hydro_discharge_avg",
             "carbon_production_unknown_avg",]

li_Legend = ["Biomass","Coal","Gas","Water","Nuclear","Oil","Photovoltaic",
             "Wind","Geothermal","PSP","Unknown"]

fig, ax = plt.subplots(2,figsize=(12, 8))
#labels = dfToPlot.index
dfToPlot = df_2018[li_ToPLot].resample("W").mean()
dfToPlot.plot(kind = "area",ax=ax[0], color = colors )
leg = ax[0].legend(loc="upper left", bbox_to_anchor=(1.025,1.05), title = "Energy source", labels = li_Legend,fontsize = 14)

ax[0].set_xlabel('')
ax[0].set_ylabel('CO$_2$ Intensity [gCO$_2$eq/kWh]', fontsize = 14)

li_ToPLot = ["carbon_origin_percent_biomass_avg","carbon_origin_percent_coal_avg",
            "carbon_origin_percent_gas_avg","carbon_origin_percent_hydro_avg",
            "carbon_origin_percent_nuclear_avg","carbon_origin_percent_oil_avg",
            "carbon_origin_percent_solar_avg","carbon_origin_percent_wind_avg",
            "carbon_origin_percent_geothermal_avg","carbon_origin_percent_hydro_discharge_avg",
             "carbon_origin_percent_unknown_avg",]


dfToPlot = df_2018[li_ToPLot].resample("W").mean()
dfToPlot.plot(kind = "area",ax=ax[1], color = colors, legend = False )

ax[1].set_xlabel('')
ax[1].set_ylabel('CO$_2$ Intensity [%]', fontsize = 14)


fig.suptitle('CO$_2$-Mix for the years 2018,2019', fontsize=20, x = 0.45)

for legobj in leg.legendHandles:
    legobj.set_linewidth(5.0)
plt.setp(ax[0].get_xticklabels(), fontsize=13)
plt.setp(ax[0].get_yticklabels(), fontsize=13)
plt.setp(ax[1].get_xticklabels(), fontsize=13)
plt.setp(ax[1].get_yticklabels(), fontsize=13)

#Move other axis out of the way
pos = ax[0].get_position()
pos.y1 = 0.95   #height
ax[0].set_position(pos)

plt.tight_layout()
plt.savefig("../Bilder/CO2-mix_Absolut_prozent_eng", bbox_inches = 'tight',pad_inches = 0.08)


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ 
stellt die stromprouktion über die Jahre aufgeteilt auf die einzelnen energieträger dar
Bin mir aber ziemlich sicher dass die Power_origin_percent auf verbrauchten strom deuten
Siehe anteil an Nuklear sowie großer Kohleanteil
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ 


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [59]:
li_Carbon_RE = ["carbon_production_biomass_avg","carbon_production_hydro_avg","carbon_production_solar_avg",
               "carbon_production_wind_avg","carbon_production_geothermal_avg","carbon_production_hydro_discharge_avg"]

li_Carbon_NRE = ["carbon_production_coal_avg","carbon_production_gas_avg","carbon_production_nuclear_avg",
                "carbon_production_oil_avg","carbon_production_unknown_avg","carbon_production_battery_discharge_avg"]


fig, ax = plt.subplots(1,2, figsize=(12,7))

sns.kdeplot( data = df_2015_2019, x = "carbon_intensity_avg", hue = "year" , ax = ax[0])
ax[0].xaxis.grid()
fig.suptitle('Verteilung der CO$_2$ Emissionen über die Jahre 2015-2020', fontsize=20)
ax[0].set_xlabel('CO$_2$ Intensität [gCO$_2$eq/kWh]', fontsize = 14)
ax[0].set_ylabel('Dichte', fontsize = 14)
ax[0].legend(loc = "upper right", labels = ["2019","2018","2017","2016","2015"], title = "Jahr", 
             fontsize = 14,facecolor='#f5f5f5', framealpha=1)

ax[0].yaxis.set_tick_params(labelsize=14)
ax[0].xaxis.set_tick_params(labelsize=14)

dfToPlot = pd.DataFrame()
dfToPlot["carbon_RE"] = df_2018[li_Carbon_RE].sum(axis=1)
dfToPlot["carbon_NRE"] = df_2018[li_Carbon_NRE].sum(axis=1)

sns.kdeplot( data = dfToPlot, x = "carbon_RE" , color = "green" , ax = ax[1])
sns.kdeplot( data = dfToPlot, x = "carbon_NRE" , color = "grey"  , ax = ax[1])

ax[1].xaxis.grid()
ax[1].set_xlabel('CO$_2$ Intensität [gCO$_2$eq/kWh]', fontsize = 14)
ax[1].set_ylabel('Dichte', fontsize = 14)
ax[1].legend(loc = "upper right", labels = ["Erneuerbar","Nicht Erneuerbar"], title = "Art der Erzeugung", fontsize = 14
            ,facecolor='#f5f5f5', framealpha=1)


ax[1].yaxis.set_tick_params(labelsize=14)
ax[1].xaxis.set_tick_params(labelsize=14)

fig.patch.set_alpha(1)
plt.savefig("../Bilder/Verteilung der CO2 Intensität", bbox_inches = 'tight',pad_inches = 0.08)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [60]:
fig, ax = plt.subplots(2,2, figsize=(12.5,8),gridspec_kw={'width_ratios': [3, 1]})

li_Carbon_RE = ["carbon_production_biomass_avg","carbon_production_hydro_avg","carbon_production_solar_avg",
               "carbon_production_wind_avg","carbon_production_geothermal_avg","carbon_production_hydro_discharge_avg"]

li_Carbon_NRE = ["carbon_production_coal_avg","carbon_production_gas_avg","carbon_production_nuclear_avg",
                "carbon_production_oil_avg","carbon_production_unknown_avg"]
               
     
li_test = ["carbon_production_nuclear_avg","carbon_production_oil_avg"]
        
li_Carbon = ["carbon_production_biomass_avg","carbon_production_hydro_avg","carbon_production_solar_avg",
               "carbon_production_wind_avg","carbon_production_hydro_discharge_avg","carbon_production_battery_discharge_avg"
                ,"carbon_production_coal_avg","carbon_production_gas_avg","carbon_production_nuclear_avg",
                "carbon_production_oil_avg","carbon_production_unknown_avg"]

li_Colors_RE = ["green", "#0088ff", "orange", "grey" , "brown", "#0000ff"]

li_Colors_NRE = ["grey", "purple", "#b4b88a", "brown" , "red"]

dfToPlot = pd.DataFrame()
for i in range(0,len(li_Carbon_RE),1):
    dfToPlot[li_Carbon_RE[i]] = df_2018[li_Carbon_RE[i]]
sns.boxplot(palette=li_Colors_RE,  data=dfToPlot, ax = ax[0][0])

sns.boxplot(y = "carbon_production_geothermal_avg",palette=["grey"], 
              data=dfToPlot, ax = ax[0][1])

ax[0][0].set_xticklabels(["Biomasse","Wasserkraft","Photovolatik","Windkraft","Geothermie","Pumpspeicher"],fontsize=14)
ax[0][1].set_xticklabels(["Geothermie"],fontsize=14)

dfToPlot = pd.DataFrame()
for i in range(0,len(li_Carbon_NRE),1):
    dfToPlot[li_Carbon_NRE[i]] = df_2018[li_Carbon_NRE[i]]
    
sns.boxplot(palette=li_Colors_NRE, data=dfToPlot, ax = ax[1][0])

dfToPlot = dfToPlot.drop(["carbon_production_coal_avg","carbon_production_gas_avg"
                          ,"carbon_production_unknown_avg"], axis = 1)

sns.boxplot(palette=["#b4b88a","brown"],
              data=dfToPlot, ax = ax[1][1])

ax[1][0].set_xticklabels(["Kohle","Gas","Nuklear","Öl","Unbekannt"],fontsize=14)
ax[1][1].set_xticklabels(["Nuklear","Öl"],fontsize=14)


for i in range(0,2,1):
    for j in range(0,2,1):
        ax[j][i].set_ylabel("CO$_2$ Intensität [gCO$_2$eq/kWh]", fontsize = 14)
        
#change width of legend color lines
for legobj in leg.legendHandles:
    legobj.set_linewidth(3.0)
    
fig.suptitle('Verteilung der CO$_2$ Emissionen für erneuerbare\n sowie nicht erneuerbare Energieträger', fontsize=20)

#Move other axis out of the way
pos = ax[0][0].get_position()
pos.y1 = 0.99    #height
ax[0][0].set_position(pos)

plt.tight_layout()
plt.savefig("../Bilder/Verteilung Energieträger", bbox_inches = 'tight',pad_inches = 0.08)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [61]:
fig, ax = plt.subplots(2,2, figsize=(12.5,8),gridspec_kw={'width_ratios': [3, 1]})


# 54  carbon_origin_percent_geothermal_avg          float64


li_Carbon_RE = ["carbon_origin_percent_biomass_avg","carbon_origin_percent_hydro_avg","carbon_origin_percent_solar_avg",
               "carbon_origin_percent_wind_avg","carbon_origin_percent_geothermal_avg","carbon_origin_percent_hydro_discharge_avg"]

li_Carbon_NRE = ["carbon_origin_percent_coal_avg","carbon_origin_percent_gas_avg","carbon_origin_percent_nuclear_avg",
                "carbon_origin_percent_oil_avg","carbon_origin_percent_unknown_avg"]
               
     
li_test = ["carbon_production_nuclear_avg","carbon_production_oil_avg"]
        
li_Carbon = ["carbon_origin_percent_biomass_avg","carbon_origin_percent_hydro_avg","carbon_origin_percent_solar_avg",
               "carbon_origin_percent_wind_avg","carbon_origin_percent_hydro_discharge_avg","carbon_production_battery_discharge_avg"
                ,"carbon_origin_percent_coal_avg","carbon_origin_percent_gas_avg","carbon_origin_percent_nuclear_avg",
                "carbon_origin_percent_oil_avg","carbon_origin_percent_unknown_avg"]

li_Colors_RE = ["green", "#0088ff", "orange", "grey" , "brown", "#0000ff"]

li_Colors_NRE = ["grey", "purple", "#b4b88a", "brown" , "red"]

dfToPlot = pd.DataFrame()
for i in range(0,len(li_Carbon_RE),1):
    dfToPlot[li_Carbon_RE[i]] = df_2018[li_Carbon_RE[i]]
sns.boxplot(palette=li_Colors_RE,  data=dfToPlot, ax = ax[0][0])

sns.boxplot(y = "carbon_origin_percent_geothermal_avg",palette=["grey"], 
              data=dfToPlot, ax = ax[0][1])

ax[0][0].set_xticklabels(["Biomasse","Wasserkraft","Photovolatik","Windkraft","Geothermie","Pumpspeicher"],fontsize=14)
ax[0][1].set_xticklabels(["Geothermie"],fontsize=14)

dfToPlot = pd.DataFrame()
for i in range(0,len(li_Carbon_NRE),1):
    dfToPlot[li_Carbon_NRE[i]] = df_2018[li_Carbon_NRE[i]]
    
sns.boxplot(palette=li_Colors_NRE, data=dfToPlot, ax = ax[1][0])

dfToPlot = dfToPlot.drop(["carbon_origin_percent_coal_avg","carbon_origin_percent_gas_avg"
                          ,"carbon_origin_percent_unknown_avg"], axis = 1)

sns.boxplot(palette=["#b4b88a","brown"], 
              data=dfToPlot, ax = ax[1][1])

ax[1][0].set_xticklabels(["Kohle","Gas","Nuklear","Öl","Unbekannt"],fontsize=14)
ax[1][1].set_xticklabels(["Nuklear","Öl"],fontsize=14)


for i in range(0,2,1):
    for j in range(0,2,1):
        ax[j][i].set_ylabel("Prozent [%]", fontsize = 14)

#change width of legend color lines
for legobj in leg.legendHandles:
    legobj.set_linewidth(3.0)
    
fig.suptitle('Verteilung der CO$_2$ Emissionen für erneuerbare\n sowie nicht erneuerbare Energieträger in Prozent', fontsize=20)

#Move other axis out of the way
pos = ax[0][0].get_position()
pos.y1 = 0.99    #height
ax[0][0].set_position(pos)

plt.tight_layout()
plt.savefig("../Bilder/Verteilung Energieträger prozent", bbox_inches = 'tight',pad_inches = 0.08)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Stromimporte

In [62]:
fig, ax = plt.subplots(2,2,  figsize=(10,6))

li_ToPlot = ["total_production_avg","total_consumption_avg","total_import_avg","total_export_avg"]
li_Color = ["green","orange","red","blue"]

li_Seasons = ["Frühling", "Sommer", "Herbst", "Winter"]
i = 0
for row in range(0,2,1):
    for col in range(0,2,1):
        for data in range(0,len(li_ToPlot),1):  
            sns.lineplot(x="hour", y=li_ToPlot[data]
                    , data=df_2018[df_2018.index.quarter == i+1] , ax = ax[row][col] 
                    ,color=li_Color[data], legend = False , ci = 0)
    
        ax[row][col].set_xlabel('')
        ax[row][col].set_ylabel('Energie [MW]', fontsize = 14)
        ax[row][col].set_title('Durchschnittlicher Tag im ' + li_Seasons[i])
        ax[row][col].set_ylim(0,10000)
        i += 1
    
    
fig.suptitle('Durchschnittliche Stromwerte\nüber die Jahre 2015-2020', fontsize=20)
leg = fig.legend(labels = ["Produktion","Verbrauch","Import","Export"], 
        bbox_to_anchor = [0.88, 0.89],prop={'size': 14}, ncol = 4,facecolor='#f5f5f5', framealpha=1)

#change width of legend color lines
for legobj in leg.legendHandles:
    legobj.set_linewidth(5.0)
    
for i in range(0,4,1):
    leg.legendHandles[i].set_color(li_Color[i]) 
    
#Move other axis out of the way
pos = ax[0][0].get_position()
pos.y1 = 1.05    #height
ax[0][0].set_position(pos)
plt.tight_layout()
plt.savefig("../Bilder/Stromwerte über einen durchschnittlichen Tag pro Quartal", bbox_inches = 'tight',pad_inches = 0.08)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [63]:

li_Months = ["Jänner","Februar","März","April","Mai","Juni","Juli","August","September","Oktober","November","Dezember"]
li_Quarter = ["Quartal: Januar - März", "Quartal: April - Juni", "Quartal: Juli - September", "Quartal: Oktober - Dezember"]
li_ToPLot = ["power_net_import_CH_avg","power_net_import_CZ_avg","power_net_import_DE_avg","power_net_import_HU_avg",
            "power_net_import_IT-NO_avg","power_net_import_SI_avg", "hour", "date", "week"]
li_Color = ["red","grey","green","blue","orange","black"]
dfToPlot = pd.DataFrame()

fig, ax = plt.subplots(4,3, figsize=(12, 12))


column = 0
row = 0


#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#Upper left: Primary Energy for one average day of the year 
for month in range(0,12,1):
    dfToPlot = df_2018[li_ToPLot].groupby(pd.Grouper(freq='H')).mean()
    dfToPlot = dfToPlot[dfToPlot.index.year == 2019]
    dfToPlot = dfToPlot[dfToPlot.index.month == month+1]
    for i in range(0,len(li_ToPLot)-3,1):  
        sns.lineplot(x="hour",y = dfToPlot[li_ToPLot[i]], data=dfToPlot , ax = ax[row][column], color = li_Color[i], ci = 0)
    
    ax[row][column].set_xticks([0,8,16])
    ax[row][column].set_xticklabels(["0","8","16"])
    ax[row][column].set_xlabel("")
    ax[row][column].set_ylabel("Ausgleichsstrom\n[MW]", fontsize = 14)
    ax[row][column].set_ylim(bottom=-1500)
    ax[row][column].set_title(li_Months[month] )
    ax[row][column].set_ylim(-1500,3000 )
    ax[row][column].get_yaxis().set_major_formatter(matplotlib.ticker.FuncFormatter(lambda x, loc: locale.format_string('%d', x, 1)))
    column += 1
    if column == 3:
        row += 1
        column = 0
  
fig.suptitle('Importe und Exporte der Nachbarländer Österreichs', fontsize=20)
#create a legend for the whole figure
leg = fig.legend(labels = ["Schweiz","Tschechien","Deutschland",
                           "Ungarn","Italien","Slowenien"], bbox_to_anchor = [0.8, 0.935],
                             prop={'size': 14},ncol = 3,facecolor='#f5f5f5', framealpha=1)
#change width of legend color lines
for legobj in leg.legendHandles:
    legobj.set_linewidth(10.0)

#Move other axis out of the way
pos = ax[0][0].get_position()
pos.y0 = 1    #height
ax[0][0].set_position(pos)
plt.tight_layout()
plt.savefig("../Bilder/Stromimporte", bbox_inches = 'tight',pad_inches = 0.08)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
df_ToPlot["import"] =  df_2018[li_ToPLot[0]]
df_ToPlot["pos"] = df_ToPlot["import"].clip(0, 15000)
df_ToPlot["neg"] = df_ToPlot["import"].clip(0, -15000)

df_ToPlot.describe()

In [ ]:
  
    
    


li_ToPLot = ["power_net_import_CH_avg","power_net_import_CZ_avg","power_net_import_DE_avg","power_net_import_HU_avg",
            "power_net_import_IT-NO_avg","power_net_import_SI_avg", "year", "date", "week"]



li_Color = ["red","grey","green","blue","orange","black"]
df_ToPlot = pd.DataFrame()
df_ToPlot["year"] = df_2018["year"]


fig, ax = plt.subplots(3,4 ,figsize=(14,10))
i = 0
for row in range(0,3,1):
    for col in range(0,4,2):
        
        df_ToPlot["import"] =  df_2018[li_ToPLot[i]]
        df_ToPlot["pos"] = df_ToPlot["import"].clip(0, 15000)
        df_ToPlot["neg"] = df_ToPlot["import"].clip(0, -15000)
        df_ToPlot["netto"] = df_ToPlot["pos"] - abs(df_ToPlot["neg"])
        
        sns.barplot(x = "year", y = "pos", data = df_ToPlot.resample("Y").sum().divide(1e6) , 
                ax = ax[row][col], palette = ["blue","blue"])
        
        sns.barplot(x = "year", y = "neg", data = df_ToPlot.resample("Y").sum().divide(1e6) , 
                ax = ax[row][col], palette= ["red","red"])
        
        if df_ToPlot["netto"].sum() > 0:
            color = "#414ac4"
        else:
            color = "#c44141"
        
        sns.barplot(x = "year", y = "netto", data = df_ToPlot.resample("Y").sum().divide(1e6) , 
                ax = ax[row][col + 1], palette= [color])
        #ax[row][col].set_ylim(-20,20)
        ax[row][col].set_title("Import/Export")
        ax[row][col].set_ylabel("Ausgleichsenergie [TWh]")
        ax[row][col].set_xlabel("")
        ax[row][col].set_xticklabels(["2018","2019"])
        
        ax[row][col+1].set_title("Netto")
        ax[row][col+1].set_ylabel("Ausgleichsenergie [TWh]")
        ax[row][col+1].set_xlabel("")
        ax[row][col+1].set_xticklabels(["2018","2019"])
        
        
        
        i += 1
        


plt.figtext(0.25,0.9, "Schweiz",fontsize = 15 , bbox=dict(fill=False, edgecolor='black', linewidth=1))

plt.figtext(0.735,0.9, "Tschechien",fontsize = 15 , bbox=dict(fill=False, edgecolor='black', linewidth=1))

plt.figtext(0.235,0.59, "Deutschland",fontsize = 15 , bbox=dict(fill=False, edgecolor='black', linewidth=1))

plt.figtext(0.75,0.59, "Ungarn",fontsize = 15 , bbox=dict(fill=False, edgecolor='black', linewidth=1))

plt.figtext(0.255,0.29, "Italien",fontsize = 15 , bbox=dict(fill=False, edgecolor='black', linewidth=1))

plt.figtext(0.745,0.29, "Slowenien",fontsize = 15 , bbox=dict(fill=False, edgecolor='black', linewidth=1))
   
fig.suptitle('Importe/Exporte der einzelnen Länder', fontsize=20)
pos = ax[0][0].get_position()
pos.y1 = pos.y1 + 0.1    #height
ax[0][0].set_position(pos)

pos = ax[1][0].get_position()
pos.y1 = pos.y1 + 0.03    #height
ax[1][0].set_position(pos)

pos = ax[2][0].get_position()
pos.y1 = pos.y1 + 0.03   #height
ax[2][0].set_position(pos)
plt.tight_layout()
plt.savefig("../Bilder/Importe_Exporte der einzelnen Länder", bbox_inches = 'tight',pad_inches = 0.08)

In [ ]:
fig, ax = plt.subplots(2,2,  figsize=(8,7))

li_ToPlot = ["carbon_intensity_avg","carbon_intensity_production_avg","carbon_intensity_import_avg"]
li_Color = ["red","green","grey"]

li_Seasons = ["Frühling", "Sommer", "Herbst", "Winter"]
i = 0
for row in range(0,2,1):
    for col in range(0,2,1):
    
        for data in range(0,len(li_ToPlot),1):  
            sns.lineplot(x="hour", y=li_ToPlot[data]
                    , data=df_2018[df_2018.index.quarter == i+1] , ax = ax[row][col] 
                    ,color=li_Color[data], legend = False , ci = 0)
    
        ax[row][col].set_xlabel('')
        ax[row][col].set_ylabel('CO$_2$ Intensität\n[gCO$_2$eq/kWh]')
        ax[row][col].set_title('Durchschnittlicher\nTag im ' + li_Seasons[i])
        ax[row][col].set_ylim(0,500)
        i +=1
    
    
fig.suptitle('Vergleich der CO$_2$-Intensitäten\nüber die Jahre 2015-2020', fontsize=20)
leg = fig.legend(labels = ["CO$_2$-Intensität","CO$_2$-Produziert","CO$_2$-Importiert"], 
        bbox_to_anchor = [0.95, 0.89],prop={'size': 14}, ncol = 3,facecolor='#f5f5f5', framealpha=1)

#change width of legend color lines
for legobj in leg.legendHandles:
    legobj.set_linewidth(5.0)
    
for i in range(0,3,1):
    leg.legendHandles[i].set_color(li_Color[i]) 
    
#Move other axis out of the way
pos = ax[0][0].get_position()
pos.y1 = 1.05    #height
ax[0][0].set_position(pos)
plt.tight_layout()
plt.savefig("../Bilder/CO2 Werte für Import_produziert_Verbraucht", bbox_inches = 'tight',pad_inches = 0.08)

In [ ]:
fig, ax = plt.subplots(2,2,  figsize=(8,7))

li_ToPlot = ["carbon_intensity_avg","carbon_intensity_production_avg","carbon_intensity_import_avg"]
li_Color = ["red","green","grey"]

li_Seasons = ["Spring", "Summer", "Autumn", "Winter"]
i = 0
for row in range(0,2,1):
    for col in range(0,2,1):
        
        for data in range(0,len(li_ToPlot),1):  
            sns.lineplot(x="hour", y=li_ToPlot[data]
                    , data=df_2018[df_2018.index.quarter == i+1] , ax = ax[row][col] 
                    ,color=li_Color[data], legend = False , ci = 0)
    
        ax[row][col] .set_xlabel('')
        ax[row][col] .set_ylabel('CO$_2$ Intensity\n[gCO$_2$eq/kWh]')
        ax[row][col] .set_title('Average day in ' + li_Seasons[i],fontsize=14)
        ax[row][col] .set_ylim(0,500)
        ax[row][col].set_xticks([0,5,10,15,20])
        ax[row][col].set_xticklabels(["0","5","10","15","20"])
        plt.setp(ax[row][col].get_xticklabels(), fontsize=13)
        plt.setp(ax[row][col].get_yticklabels(), fontsize=13)
        i += 1
    
    
fig.suptitle('Comparison of CO$_2$-Intensity', fontsize=20)
leg = fig.legend(labels = ["CO$_2$-Intensity","CO$_2$-Produced","CO$_2$-Imported"], 
        bbox_to_anchor = [0.93, 0.90],prop={'size': 14}, ncol = 3,facecolor='#f5f5f5', framealpha=1)

#change width of legend color lines
for legobj in leg.legendHandles:
    legobj.set_linewidth(5.0)
    
for i in range(0,3,1):
    leg.legendHandles[i].set_color(li_Color[i]) 
    
#Move other axis out of the way
pos = ax[0][0].get_position()
pos.y1 = 1.05    #height
ax[0][0].set_position(pos)
plt.tight_layout()
plt.savefig("../Bilder/CO2 Werte für Import_produziert_Verbraucht_eng", bbox_inches = 'tight',pad_inches = 0.08)

In [ ]:

fig, ax = plt.subplots(1, 2, figsize=(15,5))
sns.lineplot(x="hour", y="carbon_intensity_avg"
                ,hue="year", data=df_2015_2019 , ax = ax[0] , palette=["C0", "C1", "C2","C3", "C4"] )

ax[0].set_ylabel('CO$_2$-Intensität [gCO$_2$eq]')
ax[0].set_xlabel('Tag')
ax[0].set_title('CO$_2$ Emissionen über einen durchschnittlichen Tag pro Jahr')
leg =  ax[0].legend( loc='lower center',fontsize = 12, ncol = 5) 

ax[1].bar(x =  ["2015","2016","2017","2018","2019"],
          height = df_2015_2019["carbon_intensity_avg"].resample("Y").mean() )
ax[1].set_ylabel('CO$_2$-Intensität [gCO$_2$eq/kWh]')
ax[1].set_xlabel('Jahr')
ax[1].set_title('Durchschnittliche CO$_2$ Werte im Jahr')


#change width of legend color lines
for legobj in leg.legendHandles:
    legobj.set_linewidth(5.0)

plt.tight_layout()
plt.savefig("../Bilder/CO2 Emissionen über einen durchschnittlichen Tag pro Jahr")

Darstellung der monatlichen CO2 werte für die Jahre 2015-2019
Grenzwerte der OIB Richtlinie sind in rot dargestellt.


Überschreitung der Grenzwerte im Spätsommer/herbst
Abfall im frühling 2019(Vielleicht überdurchschnittlich warmer Frühling?)

Jahre 2015-2017 mitteln
OIB2019 im linekn diagramm darstellen(Jahreswert)

In [ ]:

fig, ax = plt.subplots(1, 2, figsize=(15,5))
sns.lineplot(x="month", y="carbon_intensity_avg"
                ,hue="year", data=df_2015_2019.resample("M").mean()  , ax = ax[0] 
             , palette=["C0", "C1", "C2","C3", "C4"] )
ax[0].set_ylabel('CO$_2$-Intensität [g/kWh$_e$$_l$]')
ax[0].set_xlabel('Monat')
ax[0].set_title('CO2 Emissionen verglichen mit monatlichen OIB-RL6 Grenzwerten')


ax[1].bar(x = ["2015","2016","2017","2018","2019"],
          height = df_2015_2019["carbon_intensity_avg"].resample("Y").mean() )
ax[1].set_ylabel('CO$_2$-Intensität [g/kWh$_e$$_l$]')
ax[1].set_xlabel('Jahr')
ax[1].set_title('Durchschnittliche CO$_2$ Werte im Jahr')
i = 0
while i != 11:
    #draw horizontal line
    ax[0].plot([i+1, i+2], [df_OIB_RL6["fCO2eq"][i],df_OIB_RL6["fCO2eq"][i]], 
             linewidth=3, color = "red")
    if i+1 !=11:
        #draw vertical line
        ax[0].plot([i+2, i+2], [df_OIB_RL6["fCO2eq"][i+1],df_OIB_RL6["fCO2eq"][i]],
             linewidth=3 ,color = "red")   
    i += 1
    
#invisible point for label
ax[0].plot([1, 1], [1,1], 
             linewidth=3, color = "red" , label = "OIB_RL6(2018)")    

ax[0].legend( loc='lower left',fontsize = 12) 


plt.tight_layout()

In [ ]:
#import bezier
from ipywidgets import interact, interactive, fixed, interact_manual

from IPython.display import display
import ipywidgets as widgets
from gekko import GEKKO
def f(x):
    return x

In [ ]:
l = 24

edge_length = 3

xm = np.arange(-edge_length,l+edge_length,1)
#ym = np.arange(0,l+2,1)
ym = np.random.random(l+2*edge_length)
ym[0:edge_length*2] = ym[-edge_length*2:]


print(xm)
print(ym)

In [ ]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
interact(f, x=10)

In [ ]:
slider = {}
for i in range(l):
    a = widgets.FloatSlider(min=-1.,
                            max=3.,
                            value=1.,
                            orientation="vertical",
                            title=str(i+1))
    slider[i] = a

tuple(slider.values())

In [ ]:
ui = widgets.HBox(tuple(slider.values()))
def f(*s):
    print((s))

out = widgets.interactive_output(f, slider)
display(ui, out)

In [ ]:
vals = [x.value for x in slider.values()]
vals

In [ ]:
OE_NORM = [
    [2.144,0.066,1.013,-1.969,0.352,-3.294,-3.79,-4.233,-0.731,-5.349,0.382,-0.981,0.71,-2.388],
    [3.918,2.972,3.088,1.366,2.181,-0.184,2.260,0.046,2.22,0.035,2.706,2.045,2.549,0.834],
    [7.793,7.411,7.333,6.402,6.523,4.943,6.940,5.696,7.226,6.448,7.137,6.814,6.762,5.945],
    [12.511,11.99,12.416,11.532,11.802,10.533,12.16,11.289,12.482,12.253,12.236,11.895,11.872,11.369],
    [16.576,16.013,16.652,15.874,16.220,15.175,16.462,15.772,16.841,16.84,16.666,16.362,16.306,15.881],
    [20.023,19.179,20.047,18.961,19.63,18.308,20.071,18.95,20.619,20.516,20.245,19.923,19.855,19.196],
    [21.92,20.94,21.76,20.24,21.5,19.69,21.74,20.19,22.47,22.01,21.97,21.56,21.84,20.87],
    [21.44,19.86,21.26,19.37,20.94,18.78,21.18,19.3,21.7,20.67,21.24,20.67,21.27,19.9],
    [17.6,16.44,17.45,15.93,16.94,14.91,17.29,15.73,17.57,16.63,17.37,16.92,17.28,16.13],
    [12.08,10.54,11.75,9.72,11.03,8.41,11.5,9.18,11.57,9.5,11.62,11.02,11.38,9.576],
    [7.19,5.01,6.49,3.73,5.64,2.43,5.87,2.66,5.6,2.22,5.98,4.93,5.94,3.34],
    [3.88,0.29,2.86,-1.46,2,-2.65,1.69,-3.49,1.15,-4.66,1.99,0.144,2.26,-1.89]  
]

IMporte nach Quartal vll mit CO2 als zweite achse drüberlegen

In [ ]:
fig, ax = plt.subplots(1, figsize=(10,5))




In [ ]:
fig, ax = plt.subplots(1, figsize=(10,5))
#2018: 5.114677 PJ Erzeugt Solar
#2030: 53,4 PJ Solar
#10.44 facher Ausbau


#2018: 23.02685 PJ Erzeugt Wind
#2030: 62,7 PJ Wind PJ Erzeugt Wind
#2,72 facher Ausbau


#2018: 108.040392 PJ Erzeugt Wasser
#2030: 153,8 PJ Erzeugt Wasser
#1,42 facher ausbau


PJ_to_GWH = 277.778 #GWh/PJ
li_Production = ["power_production_biomass_avg","power_production_coal_avg","power_production_gas_avg",
            "power_production_hydro_avg","power_production_nuclear_avg","power_production_oil_avg",
            "power_production_solar_avg","power_production_wind_avg","power_production_geothermal_avg",
            "power_production_unknown_avg"]


#Für Testzwecke nur mal das erste Jahr übertragen
df_GWH = pd.DataFrame(df_2018["power_production_solar_avg"].iloc[0:8759])


test = pd.DataFrame(df_2018["power_production_solar_avg"].iloc[0:8759])
test = test.divide((PJ_to_GWH*1000))   #.round()

df_GWH = df_GWH.divide((PJ_to_GWH*1000))   #.round()
#df_GWH.multiply(10.44)
df_GWH["power_production_solar_avg"] = df_GWH["power_production_solar_avg"] * 10.44
print("Summe 2030: ",df_GWH["power_production_solar_avg"].sum() )

print(df_GWH["power_production_solar_avg"].iloc[1000])
print(test["power_production_solar_avg"].iloc[1000])

test["power_production_solar_avg"].resample("W").mean().plot()
df_GWH["power_production_solar_avg"].resample("W").mean().plot()
ax.legend(loc="upper left", bbox_to_anchor=(1,1))

plt.tight_layout()

In [ ]:
df_2018.info(verbose = True)

In [ ]:
#plt.close('all')
